In [1]:
pip install langchain langchain-openai langchain-community langchain-text-splitters sentence-transformers pypdfium2 chromadb langchain-chroma

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
pip install -U langchain langchain-community langchain-openai

   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---------------- ----------------------- 1.0/2.5 MB 6.5 MB/s eta 0:00:01
   ---------------------------------------- 2.5/2.5 MB 6.9 MB/s eta 0:00:00
  Attempting uninstall: langchain-openai
    Found existing installation: langchain-openai 0.3.16
    Uninstalling langchain-openai-0.3.16:
      Successfully uninstalled langchain-openai-0.3.16
  Attempting uninstall: langchain-community
    Found existing installation: langchain-community 0.3.21
    Uninstalling langchain-community-0.3.21:
      Successfully uninstalled langchain-community-0.3.21
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


#### 임베딩 모델

In [4]:
from langchain.embeddings import HuggingFaceEmbeddings

ko_embedding = HuggingFaceEmbeddings(
    model_name="jhgan/ko-sroberta-multitask",  # 한국어 특화 모델
    model_kwargs={"device": "cpu"}             # 또는 "cuda" 사용 가능
)

C:\Users\문선장\AppData\Local\Temp\ipykernel_30452\725957186.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  ko_embedding = HuggingFaceEmbeddings(
C:\Users\문선장\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [50]:
import shutil
import os
from pathlib import Path
import torch

# 전체 Chroma DB 삭제 (기본 디렉토리와 시스템 캐시 포함)
shutil.rmtree("./chroma_db", ignore_errors=True)
shutil.rmtree(Path.home() / ".chromadb", ignore_errors=True)

import os
import json
from langchain_openai import OpenAIEmbeddings
from langchain_core.documents import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
import chromadb
from langchain.document_loaders import PyPDFLoader

os.environ["OPENAI_API_KEY"] = "키"

embeddings_model  =ko_embedding
text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=50)
client = chromadb.PersistentClient(path="./chroma_db")

def load_json_documents(json_files):
    docs = []
    for file in json_files:
        with open(file, "r", encoding="utf-8") as f:
            data = json.load(f)

        #시나리오
        for item in data:
            if "Text" in item and "Completion" in item:
                docs.append(Document(
                    page_content=f"질문: {item['Text']}\n답변: {item['Completion']}",
                    metadata={
                        "카테고리": item.get("카테고리", "")
                    }
                ))

            # 수강이력
            if "lecture_name" in item and "student_id" in item:
                content = (
                    f"학번: {item['student_id']}\n"
                    f"강의명: {item['lecture_name']}\n"
                    f"학정번호: {item['lecture_id']}\n"
                    f"개설 학과: {item['department_offered']}\n"
                    f"이수 구분: {item['lecture_course_type']}\n"
                    f"학점: {item['lecture_credit']}학점\n"
                    f"성적: {item['lecutre_grade']}"
                )
                if "retake_or_delete_status" in item:
                    content += f"\n재수강/삭제 여부: {item['retake_or_delete_status']}"
                if "retake_status" in item:
                    content += f"\n재수강 여부: {item['retake_status']}"

                docs.append(Document(
                    page_content=content,
                    metadata={
                        "lecture_name": item["lecture_name"],
                        "course_type": item["lecture_course_type"]
                    }
                ))

            # 강의 탐색
            if "lecture_id" in item and "lecture_name" in item and "student_id" not in item:
                content = (
                    f"학정번호: {item.get('lecture_id', '')}\n"
                    f"강의명: {item.get('lecture_name', '')}\n"
                    f"강의평점: {item.get('lecture_ratings', '')}\n"
                    f"과제: {item.get('lecture_homework', '')}\n"
                    f"팀플: {item.get('lecture_team', '')}\n"
                    f"성적평가정도: {item.get('lecutre_grade', '')}\n"
                    f"출결 방식: {item.get('lecutre_attendance', '')}\n"
                    f"시험 횟수: {item.get('lecutre_test', '')}\n"
                    f"시험 방식: {item.get('lecture_testinform', '')}\n"
                    f"전공 학점: {item.get('credits_major', '없음')}\n"
                    f"교양 학점: {item.get('credits_general', '없음')}\n"
                    f"총 학점: {item.get('credits_total', '없음')}\n"
                    f"교수명: {item.get('lecture_professorname', '')}\n"
                    f"수업 시간: {item.get('lecture_time', '')}\n"
                    f"강의 유형: {item.get('lecture_course_type', '')}\n"
                    f"학점: {item.get('lecture_hours', '')}시간\n"
                    f"학기: {item.get('lecture_semester', '')}학기\n"
                    f"강의 설명: {item.get('lecture_inform', '')}"
                    f"영역: {item.get('lecture_domain', '')}\n"
                )
                docs.append(Document(
                    page_content=content,
                    metadata={
                        "lecture_id": item.get("lecture_id", ""),
                        "lecture_name": item.get("lecture_name", ""),
                        "lecture_ratings": item.get("lecture_ratings", ""),
                        "lecture_team": item.get("lecture_team", ""),
                        "lecutre_grade": item.get("lecutre_grade", ""),
                        "professor": item.get("lecture_professorname", "")
                    }
                ))
    return docs

user_datasets = {
    "kim": ["data/kw_chatbot_data - 김브티_수강이력.json"],
    "hong": ["data/kw_chatbot_data - 홍데사_수강이력.json"]
}

task_datasets = {
    "lecture_search": [
        "data/강의탐색.json",
        "data/kw_chatbot_data - Student.json",
        "data/kw_chatbot_data - 강의 평점.json",
        "data/kw_chatbot_data - 강의계획서.json",
        "data/kw_chatbot_data - 수강신청자료집.json",
        "data/kw_chatbot_data - 커리큘럼(DS).json",
        "data/kw_chatbot_data - 커리큘럼(VT).json"
    ],
    "career_counsel": [
        "data/진로상담.json",
        "data/kw_chatbot_data - 강의계획서.json",        
        "data/kw_chatbot_data - 김브티_성적.json",
        "data/kw_chatbot_data - 김브티_수강이력.json",
        "data/kw_chatbot_data - 수강신청자료집.json",
        "data/kw_chatbot_data - 커리큘럼(DS).json",
        "data/kw_chatbot_data - 커리큘럼(VT).json",
        "data/kw_chatbot_data - 홍데사_성적.json",
        "data/kw_chatbot_data - 홍데사_수강이력.json"],
    
    "academic_status": [
        "data/학습현황.json",
        "data/kw_chatbot_data - 김브티_성적.json",
        "data/kw_chatbot_data - 김브티_수강이력.json",
        "data/kw_chatbot_data - 수강신청자료집.json",
        "data/kw_chatbot_data - 강의 평점.json",        
        "data/kw_chatbot_data - 홍데사_성적.json",
        "data/kw_chatbot_data - 홍데사_수강이력.json",
        "data/lecture_domain.json"
        ]
}

# 사용자별 컬렉션
for user_id, files in user_datasets.items():
    collection_name = f"lecture_search_{user_id}"
    try:
        client.delete_collection(name=collection_name)
    except:
        pass
    docs = load_json_documents(files)
    split_docs = text_splitter.split_documents(docs)
    texts = [doc.page_content for doc in split_docs]
    embeddings = embeddings_model.embed_documents(texts)
    collection = client.create_collection(name=collection_name)
    collection.add(
        documents=texts,
        embeddings=embeddings,
        ids=[f"{collection_name}_{i}" for i in range(len(texts))]
    )
    print(f"[{user_id}] 임베딩 완료 ({len(embeddings)}건)")

# 기능별 컬렉션
for task, files in task_datasets.items():
    try:
        client.delete_collection(name=task)
    except:
        pass
    docs = load_json_documents(files)
    split_docs = text_splitter.split_documents(docs)
    texts = [doc.page_content for doc in split_docs]
    embeddings = embeddings_model.embed_documents(texts)
    collection = client.create_collection(name=task)
    collection.add(
        documents=texts,
        embeddings=embeddings,
        ids=[f"{task}_{i}" for i in range(len(texts))]
    )
    print(f"[{task}] 임베딩 완료 ({len(embeddings)}건)")
    
# PDF 문서 임베딩 추가
target_collections = ["lecture_search", "career_counsel", "academic_status"]

from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = PyPDFLoader("data/수강신청_자료집_전체(2025-1)v4.pdf")
pages = loader.load_and_split()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=100)
docs = text_splitter.split_documents(pages)
texts = [doc.page_content for doc in docs]
embeddings = embeddings_model.embed_documents(texts)

client = chromadb.PersistentClient(path="./chroma_db")

#컬렉션에 추가
for name in target_collections:
    collection = client.get_or_create_collection(name=name)
    collection.add(
        documents=texts,
        embeddings=embeddings,
        ids=[f"{name}_pdf_{i}" for i in range(len(texts))]
    )
    print(f" '{name}' PDF 임베딩 {len(texts)}개 추가 완료")

[kim] 임베딩 완료 (24건)
[hong] 임베딩 완료 (24건)
[lecture_search] 임베딩 완료 (648건)
[career_counsel] 임베딩 완료 (682건)
[academic_status] 임베딩 완료 (641건)
 'lecture_search' PDF 임베딩 259개 추가 완료
 'career_counsel' PDF 임베딩 259개 추가 완료
 'academic_status' PDF 임베딩 259개 추가 완료


#### 기능 구현 함수

In [51]:
import json
import re
import random
import itertools
from collections import defaultdict, Counter

# LangChain 컬렉션 이름 결정 함수
def get_collection_name(user_input: str, user_id: str) -> str:
    q = user_input.lower()
    if "진로" in q or "상담" in q:
        return "career_counsel"
    elif "이수" in q or "학점" in q or "현황" in q or "평균" in q:
        return "academic_status"
    return f"lecture_search_{user_id}"

# 입력 분류 함수
def classify_function(user_input: str) -> str:
    if "강의" in user_input or "탐색" in user_input:
        return "lecture_search"
    elif "진로" in user_input or "상담" in user_input:
        return "career_counsel"
    elif "학업" in user_input or "현황" in user_input:
        return "academic_status"
    else:
        return "lecture_search"

# 캐시: 팀플 조건에 따라 결과 재사용
TEAM_CACHE = {}

# JSON 파일에서 팀플 조건과 일치하는 강의 필터링
def filter_lectures_by_team(json_files, team_condition):
    if team_condition in TEAM_CACHE:
        return TEAM_CACHE[team_condition]

    result = []
    for file in json_files:
        with open(file, "r", encoding="utf-8") as f:
            data = json.load(f)
        for item in data:
            team_value = item.get("lecture_team", "").strip()
            lecture_id = item.get("lecture_id")
            lecture_name = item.get("lecture_name")

            if not lecture_id or not lecture_name:
                continue

            if team_condition == "없음" and team_value in ["없음", "0"]:
                result.append({"lecture_id": lecture_id, "lecture_name": lecture_name})
            elif team_condition == "보통" and team_value in ["보통", "적은"]:
                result.append({"lecture_id": lecture_id, "lecture_name": lecture_name})
            elif team_condition == "많은" and team_value in ["많은", "많음"]:
                result.append({"lecture_id": lecture_id, "lecture_name": lecture_name})

    TEAM_CACHE[team_condition] = result
    return result

# 팀플 응답 메시지 포맷
def format_team_project_response(team_condition: str, lectures: list):
    if not lectures:
        return f"팀플이 {team_condition} 강의를 찾을 수 없습니다."

    condition_map = {
        "없음": "없는",
        "보통": "적은",
        "많은": "많은"
    }
    label = condition_map.get(team_condition, team_condition)
    header = f"팀플이 {label} 강의는 다음과 같습니다:"
    body = "\n".join(
        f"{i+1}. {lec['lecture_name']} (학정번호: {lec['lecture_id']})"
        for i, lec in enumerate(lectures)
    )
    return f"{header}\n{body}"

# JSON 내 팀플 조건 처리 핸들러
def handle_team_project_query(user_input, json_path):
    condition = normalize_team_condition(user_input)
    if not condition:
        return None

    filtered_lectures = filter_lectures_by_team([json_path], condition)
    seen = set()
    unique_lectures = []
    for lec in filtered_lectures:
        if lec["lecture_name"] not in seen:
            seen.add(lec["lecture_name"])
            unique_lectures.append(lec)

    return format_team_project_response(condition, unique_lectures)

# 학점 및 교과구분에 따른 강의 필터링
def get_lectures_by_credit_and_type(user_input: str):
    credit_match = re.search(r'(\d)\s*학점', user_input)
    if not credit_match:
        return "몇 학점짜리 수업을 원하시는지 알려주세요. 예: '2학점 교양 수업 알려줘'"
    credit = int(credit_match.group(1))

    # 수업 유형 필터링
    course_types = []
    if "전필" in user_input:
        course_types = ["전필"]
    elif "전선" in user_input:
        course_types = ["전선"]
    elif "교필" in user_input:
        course_types = ["교필"]
    elif "교선" in user_input:
        course_types = ["교선"]
    elif "전공" in user_input:
        course_types = ["전필", "전선"]
    elif "교양" in user_input:
        course_types = ["교필", "교선"]
    else:
        course_types = ["전필", "전선", "교필", "교선"]

    count_match = re.search(r'(\d+)\s*개', user_input)
    limit = int(count_match.group(1)) if count_match else None

    with open("./data/kw_chatbot_data - 수강신청자료집.json", "r", encoding="utf-8") as f:
        course_data = json.load(f)

    results = []
    seen = set()
    for item in course_data:
        course_type = item.get("lecture_course_type", "").strip()
        credit_val = item.get("lecture_hours") or item.get("lecture_credit")
        if not credit_val:
            continue
        try:
            credit_val = int(str(credit_val).strip())
        except:
            continue

        professor = item.get("lecture_professor", "").strip()
        lecture_name = item.get("lecture_name", "").strip()
        key = (lecture_name, professor)

        if course_type in course_types and credit_val == credit and key not in seen:
            seen.add(key)
            results.append(f"{lecture_name} (학정번호: {item['lecture_id']}) - {professor or '교수명 없음'}")

    if not results:
        return f"{credit}학점짜리 {', '.join(course_types)} 과목을 찾을 수 없습니다."

    if limit:
        random.shuffle(results)
        results = results[:limit]

    header = f"{credit}학점짜리 {', '.join(course_types)} 과목 목록입니다:"
    lines = [f"{i+1}. {lec}" for i, lec in enumerate(results)]
    return header + "\n" + "\n".join(lines)

# 성적 → 평점 매핑
GRADE_TO_POINT = {
    "A+": 4.5, "A0": 4.0, "B+": 3.5, "B0": 3.0,
    "C+": 2.5, "C0": 2.0, "F": 0.0
}

# 수강이력 불러오기 (ID → 파일명 매핑)
def load_student_data(user_id: str):
    filename = f"data/kw_chatbot_data - {'김브티' if user_id == 'kim' else '홍데사'}_수강이력.json"
    with open(filename, encoding='utf-8') as f:
        return json.load(f)

# 재수강 횟수 계산
def count_retake_courses(user_id: str):
    data = load_student_data(user_id)
    course_count = defaultdict(int)
    for record in data:
        if record.get("retake_status") == "R":
            course_count[record["lecture_name"]] += 1
    return course_count

# 특정 과목의 성적 조회
def get_course_grade(user_id: str, course_name: str):
    data = load_student_data(user_id)
    for record in data:
        if record["lecture_name"].replace(" ", "") == course_name.replace(" ", ""):
            return record["lecutre_grade"]
    return None

# 사용자 이름 매핑
USER_NAME = {
    "kim": "김브티",
    "hong": "홍데사"
}

# 평균 평점 계산 함수
# GPA 계산 함수 (재수강 반영 여부 및 전공 필터 포함)
def get_gpa_with_retake(user_id: str, exclude_pre_retake: bool = False, course_type: str = None):
    data = load_student_data(user_id)
    seen = {}
    credits, total = 0, 0
    for record in data:
        key = record['lecture_id']
        grade = record['lecutre_grade']
        credit = record['lecture_credit']
        ctype = record['lecture_course_type']

        if course_type:
            if course_type == "전공" and not ctype.startswith(("전필", "전선")):
                continue
            elif course_type != "전공" and not ctype.startswith(course_type):
                continue

        if exclude_pre_retake and record.get('retake_or_delete_status') == 'Y':
            continue

        if record.get('retake_status') == 'R':
            seen[key] = (GRADE_TO_POINT[grade], credit)
        elif key not in seen:
            seen[key] = (GRADE_TO_POINT[grade], credit)

    for point, credit in seen.values():
        credits += credit
        total += point * credit

    if credits == 0:
        return "계산할 수 있는 학점이 없습니다."
    name = USER_NAME.get(user_id, user_id)
    return f"{name}님의 평균 평점은 {round(total / credits, 2)}입니다."

# 재수강한 과목 리스트 반환
def get_retake_course_names(user_id: str):
    data = load_student_data(user_id)
    names = [r['lecture_name'] for r in data if r.get('retake_or_delete_status') == 'Y']
    return names

# 재수강한 과목 개수 반환
def get_retake_course_count(user_id: str):
    return len(get_retake_course_names(user_id))

# 특정 성적을 받은 과목 조회 함수
def get_subjects_by_grade(user_id: str, target_grade: str):
    data = load_student_data(user_id)
    return [r['lecture_name'] for r in data if r['lecutre_grade'] == target_grade]

# 특정 과목의 최종 성적 반환 함수
def get_final_grade_for_subject(user_id: str, subject_name: str):
    data = load_student_data(user_id)
    matches = [r for r in data if r['lecture_name'] == subject_name]
    if not matches:
        return None
    return matches[-1]['lecutre_grade']

# 재수강 횟수 과목별로 반환
def get_retake_course_counts(user_id: str):
    data = load_student_data(user_id)
    names = [r['lecture_name'] for r in data if r.get('retake_status') == 'R']
    return Counter(names)

# 재수강 가능 과목 조회 함수 (정확하게 재수강 제외 기준 반영)
def get_possible_retake_gpa(user_id: str, focus: str = "all", list_only: bool = False, fixed_grade: str = None):
    data = load_student_data(user_id)
    seen = {}
    excluded = {}

    # 재수강 후 B0 이상 받은 과목 제외
    for record in data:
        key = record['lecture_id']
        grade = record['lecutre_grade']
        if record.get('retake_status') == 'R' and GRADE_TO_POINT.get(grade, 0) >= 3.0:
            excluded[key] = True

    for record in data:
        key = record['lecture_id']
        grade = record['lecutre_grade']
        ctype = record['lecture_course_type']

        if record.get('retake_or_delete_status') == 'Y': continue
        if key in excluded: continue
        if focus == "major" and not ctype.startswith("전공"): continue
        if GRADE_TO_POINT.get(grade, 5) > 2.5: continue
        if key not in seen:
            seen[key] = (GRADE_TO_POINT[grade], record['lecture_credit'], record['lecture_name'])

    if list_only:
        if not seen:
            return f"지금 재수강 가능한 {'전공 ' if focus == 'major' else ''}과목은 없습니다."
        return f"재수강 가능한 {'전공 ' if focus == 'major' else ''}과목: " + ", ".join([v[2] for v in seen.values()])

def check_graduation_required_courses(user_id: str, required_json_path: str):
    # 필수 과목 목록 추출
    with open(required_json_path, encoding='utf-8') as f:
        all_courses = json.load(f)

    required = {
        (item['lecture_name'], item['lecture_course_type'])
        for item in all_courses
        if item.get('lecture_course_type') in ["교필", "전필"]
    }

    # 사용자가 이수한 과목 목록
    data = load_student_data(user_id)
    taken = {
        (record['lecture_name'], record['lecture_course_type'])
        for record in data
        if record.get("retake_or_delete_status") != "Y"
    }

    taken_required = sorted(required & taken, key=lambda x: x[1])  # 이수한 필수
    missed_required = sorted(required - taken, key=lambda x: x[1])  # 미이수 필수

    def format_list(items):
        result = defaultdict(list)
        for name, typ in items:
            result[typ].append(name)
        return result

    taken_fmt = format_list(taken_required)
    missed_fmt = format_list(missed_required)

    response_parts = []
    if taken_fmt:
        parts = [f"{typ} 과목인 {', '.join(names)}" for typ, names in taken_fmt.items()]
        response_parts.append("졸업 필수 과목 중에서 " + "과 " .join(parts) + "를 이수하셨고")
    if missed_fmt:
        parts = [f"{typ} 과목인 {', '.join(names)}" for typ, names in missed_fmt.items()]
        response_parts.append(" ".join(parts) + "를 이수하지 않았습니다.")
    if not missed_required:
        response_parts = ["졸업 필수 과목을 모두 이수하셨습니다."]
    return " ".join(response_parts)

def get_remaining_credits(user_id: str, graduation_total: int = 133):
    data = load_student_data(user_id)
    total_credits = sum(
        record['lecture_credit']
        for record in data
        if record.get('retake_or_delete_status') != "Y"
    )
    remaining = graduation_total - total_credits
    name = USER_NAME.get(user_id, user_id)
    return f"{name}님의 현재까지 이수한 학점은 {total_credits}학점이며, 졸업을 위해 {remaining}학점이 더 필요합니다."

def calculate_gpa_from_raw_split(user_id: str) -> dict:
    data = load_student_data(user_id)
    name = USER_NAME.get(user_id, user_id)

    total_points, total_credits = 0, 0
    major_points, major_credits = 0, 0

    for record in data:
        if record.get('retake_or_delete_status') == 'Y':
            continue

        grade = record.get('lecutre_grade')
        point = GRADE_TO_POINT.get(grade, None)
        credit = record.get('lecture_credit', 0)
        ctype = record.get('lecture_course_type', '')

        if point is None or not isinstance(credit, (int, float)):
            continue

        # 전체 평점 누적
        total_points += point * credit
        total_credits += credit

        # 전공 평점 누적 (전필 또는 전선만)
        if ctype.startswith("전필") or ctype.startswith("전선"):
            major_points += point * credit
            major_credits += credit

    result = {}
    if total_credits > 0:
        result["전체"] = round(total_points / total_credits, 2)
    if major_credits > 0:
        result["전공"] = round(major_points / major_credits, 2)
    else:
        result["전공"] = None  # 또는 "전공 과목 없음"

    return result

# 성적 향상 가정 시 GPA 시뮬레이션 함수 개선
def simulate_retake_gpa(user_id: str, fixed_grade: str = None, focus: str = "all"):
    if fixed_grade == "A+":
        return "재수강 시 받을 수 있는 최고학점은 A0입니다."

    data = load_student_data(user_id)
    seen = {}
    excluded = set()

    for record in data:
        key = record['lecture_id']
        grade = record['lecutre_grade']
        if record.get('retake_status') == 'R' and GRADE_TO_POINT.get(grade, 0) >= 3.0:
            excluded.add(key)
        if record.get('retake_or_delete_status') == 'Y':
            excluded.add(key)

    replace_targets = {}
    base = {}
    for record in data:
        key = record['lecture_id']
        grade = record['lecutre_grade']
        credit = record['lecture_credit']
        ctype = record['lecture_course_type']
        name = record['lecture_name']

        if key in excluded:
            continue
        if GRADE_TO_POINT.get(grade, 5) > 2.5:
            continue
        if focus == 'major' and not ctype.startswith("전공"):
            continue
        if focus == 'liberal' and ctype.startswith("전공"):
            continue

        if key not in replace_targets:
            replace_targets[key] = (GRADE_TO_POINT[grade], credit, name)

    for record in data:
        key = record['lecture_id']
        if record.get('retake_or_delete_status') == 'Y':
            continue
        base[key] = (GRADE_TO_POINT.get(record['lecutre_grade'], 0.0), record['lecture_credit'])

    grade_options = [fixed_grade] if fixed_grade else ["A0", "B+", "B0", "C+", "C0"]
    result_lines = []

    for g in grade_options:
        modified = base.copy()
        label = []
        for key, (_, credit, name) in replace_targets.items():
            modified[key] = (GRADE_TO_POINT[g], credit)
            label.append(f"{name}: {g}")
        total_credits = sum(c for _, c in modified.values())
        total_points = sum(p * c for p, c in modified.values())
        avg = round(total_points / total_credits, 2)
        result_lines.append(f"{' / '.join(label)} → 예상 평점: {avg}")

    return "\n".join(result_lines)

def get_retake_reason_summary(user_id: str) -> str:
    data = load_student_data(user_id)
    results = []

    for record in data:
        name = record.get('lecture_name')
        grade = record.get('lecutre_grade')
        course_type = record.get('lecture_course_type', '')
        is_retake = record.get('retake_status') == 'R'
        is_dropped = record.get('retake_or_delete_status') == 'Y'

        if is_dropped:
            continue
        if not course_type.startswith("전필"):
            continue
        if grade not in ["C+", "C0", "F"]:
            continue

        msg = f"{name}에서 {grade}를 받으셨습니다"
        if is_retake:
            msg += ", 재수강하셨습니다"
        results.append(msg)

    if not results:
        return "성적이 낮은 전공 필수 과목은 없습니다."

    return "전공 필수 과목 중 성적이 낮은 과목은 " + " / ".join(results) + "."

    # GPA simulation part
    original_data = load_student_data(user_id)
    base = {}
    for r in original_data:
        key = r['lecture_id']
        if r.get('retake_or_delete_status') == 'Y': continue
        base[key] = (GRADE_TO_POINT.get(r['lecutre_grade'], 0.0), r['lecture_credit'])

    replace_targets = list(seen.items())
    grade_options = [fixed_grade] if fixed_grade else list(GRADE_TO_POINT.keys())
    responses = []

    for combo in itertools.product(grade_options, repeat=len(replace_targets)):
        modified = base.copy()
        label = []
        for (key, (_, credit, name)), g in zip(replace_targets, combo):
            modified[key] = (GRADE_TO_POINT[g], credit)
            label.append(f"{name}: {g}")
        total_credits = sum(c for _, c in modified.values())
        total_points = sum(p * c for p, c in modified.values())
        avg = round(total_points / total_credits, 2)
        responses.append(f"{' / '.join(label)} → 예상 평점: {avg}")

    return "\n".join(responses)

#### ChromaDB 검색

In [57]:
import json
import re
import random
import itertools
from collections import defaultdict, Counter
from langchain_chroma import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from functools import lru_cache
from difflib import SequenceMatcher

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.3)

# 재수강 가능 과목 필터링 함수 (전공/교양 구분 포함)
def get_filtered_retake_courses(user_id: str, focus: str = "all") -> list:
    data = load_student_data(user_id)
    seen = {}
    excluded = set()

    # 재수강했고 성적이 B0 이상이면 제외
    for record in data:
        key = record['lecture_id']
        grade = record['lecutre_grade']
        if record.get('retake_status') == 'R' and GRADE_TO_POINT.get(grade, 0) >= 3.0:
            excluded.add(key)
        if record.get('retake_or_delete_status') == 'Y':
            excluded.add(key)

     # 재수강 조건에 부합하는 과목 수집
    for record in data:
        key = record['lecture_id']
        grade = record['lecutre_grade']
        ctype = record['lecture_course_type']
        name = record['lecture_name']

        if key in excluded:
            continue
        if GRADE_TO_POINT.get(grade, 5) > 2.5:
            continue
        if focus == 'major' and not ctype.startswith("전공"):
            continue
        if focus == 'liberal' and ctype.startswith("전공"):
            continue
        if key not in seen:
            seen[key] = name

    return list(seen.values())

# 팀플 조건별 응답 포맷 구성
def format_team_project_response(team_condition: str, lectures: list):
    if not lectures:
        return f"팀플이 {team_condition} 강의를 찾을 수 없습니다."

    condition_map = {
    "없음": "없는",
    "보통": "적은",
    "많은": "많은"
    }
    label = condition_map.get(team_condition, team_condition)
    header = f"팀플이 {label} 강의는 다음과 같습니다:"
    body = "\n".join(
        f"{i+1}. {lec['lecture_name']} (학정번호: {lec['lecture_id']})"
        for i, lec in enumerate(lectures)
    )
    return f"{header}\n{body}"

# 조별과제/팀플 조건 파싱 및 정규화
def normalize_team_condition(user_input: str) -> str | None:
    condition_map = {
        "없음": ["팀플 없는", "팀플이 없는", "조별과제 없는", "조별과제가 없는"],
        "보통": ["팀플 보통", "팀플이 보통", "조별과제 보통", "조별과제가 보통", "팀플 적은", "팀플이 적은", "조별과제 적은", "조별과제가 적은"],
        "많은": ["팀플 많은", "팀플이 많은", "조별과제 많은", "조별과제가 많은", "팀플 많음", "조별과제 많음"]
    }
    for condition, variants in condition_map.items():
        if any(v in user_input for v in variants):
            return condition
    return None

# Load scenario data
SCENARIO_FILES = ["./data/강의탐색.json", "./data/진로상담.json", "./data/학습현황.json"]

def load_scenario_data():
    data = []
    for path in SCENARIO_FILES:
        try:
            with open(path, encoding="utf-8") as f:
                data.extend(json.load(f))
        except FileNotFoundError:
            print(f"⚠️ 파일 누락: {path}")
    return data

SCENARIOS = load_scenario_data()

def match_scenario_answer(user_input: str, threshold: float = 0.83) -> str | None:
    norm_input = user_input.replace(" ", "").lower()
    best_score = 0
    best_answer = None

    for item in SCENARIOS:
        q = item.get("Text", "")
        a = item.get("Completion", "")
        q_norm = q.replace(" ", "").lower()
        score = SequenceMatcher(None, norm_input, q_norm).ratio()
        if score > best_score:
            best_score = score
            best_answer = a

    return best_answer if best_score >= threshold else None

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=
        """❗ 반드시 아래 문서 내 정보만 사용해서 답변하세요.
            ❌ 문서에 없는 정보는 생성하지 마세요.
            ❌ 유추, 추론, 보완, 일반 상식 사용 금지.
            
                - 두 개 이상인 경우에는 **모두 나열**하세요.
                - 조건에 맞는 강의가 없는 경우, "추천할 수 있는 과목이 없습니다."라고 하세요.
                - "수강이력.json"에 있는 과목은 제외하고 추천하세요.
                - 000교수님 강의 추천해줘 → 질문에는 "수강신청자료집.json"에서 lecture_professor항목에서 검색하여 대답하세요.
                - 반드시 "수강신청 자료집.json"과 "수강신청_자료집_전체.pdf" 파일만 사용하세요.
                - 교수님 이름을 헷갈리지 마고 정확히 적어주세요.
                - 예: "OOO 교수님 강의 추천해줘" → "OOO 교수님의 강의 OOO 추천 드립니다."

            ✅ **성적 응답**
                - 성적 표현은 반드시 데이터에 있는 A+, A0, B+, B0, C+, C0, F 등등으로만 응답하세요.
                - 예: "A+"를 물으면 A+만 보여주고, A0는 포함하지 마세요.
                - 유사 성적 포함, 평균값 추론 등은 절대 하지 마세요.
                - 졸업 학점 기준은 "2022학년도 입학자" 기준으로 133학점 입니다. 이에 맞춰서 졸업 가능 여부를 판단하세요.

            ✅ **강의 평가 응답**
                - 조별과제, 팀플 많음 여부, 시험 성적 너그러움 여부에 대한 질문은 **반드시 강의 평점.json 파일만 사용**해서 답하세요.

            ✅ **답변 형식**
                - 반드시 아래 말투와 형식을 따르세요:
                - "자료구조 성적이 뭐야?" → "자료구조의 성적은 B+입니다."
                - "내가 A+ 받은 과목 알려줘" → "OOO, OOO 과목에서 A+를 받으셨습니다."
                - "객체지향프로그래밍이 전선이지?" → "네, 객체지향프로그래밍은 전필입니다."
                - "전체 평균이 어떻게 돼?" → "전체 성적 평균은 3.58입니다."
                - 답변 앞에 "답변:", "안녕하세요", "요약하자면" 등의 말은 **절대 붙이지 마세요.**
                - 추가 질문 권유, 응원의 말은 절대 하지마세요.
                
                당신은 광운대학교 학사정보를 기반으로 한 진로상담 챗봇입니다. 다음 세 가지 기능을 수행합니다:

                    1. 강의탐색: 사용자에게 적절한 강의나 과목을 추천합니다.
                    2. 학습현황: 사용자의 성적 및 수강 과목 정보를 요약하거나 조회합니다.
                    3. 진로상담: 사용자 성적, 이수 과목 등을 바탕으로 진로 관련 질문에 응답합니다.

                    ❗ 반드시 아래 지침을 따르세요:

                    ✅ **데이터 제한**
                        - "강의탐색" 및 "학습현황" 기능은 반드시 제공된 pdf 및 json 내 데이터만을 사용하여 응답하세요.
                        - 데이터에 없는 정보는 절대 생성하거나 유추하지 마세요.
                        - 없으면 "해당 정보는 제공되지 않았습니다"라고 명확히 말하세요.

                    ✅ **강의 추천**
                        - 조건에 맞는 강의가 하나라면 **무 말, 불필요한 부연은 금지합니다.
                        - 광운대학교 외의 다른 학교에 대한 정보는 절대 제공하지 마세요.

                    📄 문서: {context}
                    💬 질문: {question}

                    지침을 반드시 지켜서 **자연스럽고 간결하게 말문을 여세요.**

            문서 내용은 다음과 같은 형식을 따릅니다:
    
                질문: ...
                답변: ...
                
                사용자 질문과 가장 유사한 질문이 문서에 있을 경우,
                해당 질문과 함께 적혀 있는 **답변만 그대로 출력**하세요.
                
                절대 새로운 문장을 생성하거나 다른 정보를 섞지 마세요."""

)

vectorstore = Chroma(
    persist_directory="./chroma_db",
    collection_name="lecture_search",
    embedding_function=embeddings_model
)
qa = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectorstore.as_retriever(search_kwargs={"k": 20}),
    chain_type="stuff",
    chain_type_kwargs={"prompt": prompt}
)
    
# 자연어 질의 응답 처리 함수
def answer_query(user_input: str, user_id: str = "kim"):
    lowered = user_input.lower()
    normalized = lowered.replace(" ", "")

    collection_name = get_collection_name(user_input, user_id)
    
    # 졸업논문/프로젝트 관련 질의 선처리
    graduation_keywords = ["졸업논문", "졸업프로젝트", "졸프", "졸작", "졸업과제"]
    
    # 시기 관련 질의는 가장 먼저 처리
    if all(k in lowered for k in ["졸업", "논문", "프로젝트"]) and any(w in lowered for w in ["언제", "언제부터", "시기", "몇학년", "준비", "시작"]):
        return ("빠르면 3학년부터 준비하는 경우도 있지만, 대부분 3학년 2학기부터 팀을 구한 후 4학년 때 프로젝트를 진행합니다. 논문의 경우도 대부분 4학년 때 준비합니다.")
    
    # 필수 여부
    if all(k in lowered for k in ["졸업", "논문", "프로젝트"]) and any(w in lowered for w in ["필수", "의무", "꼭", "반드시", "필요", "해야", "해야해"]):
        return "졸업하기 위해서는 졸업 논문 혹은 졸업 프로젝트 둘 중 하나는 필수입니다."
    
    # 기타 졸프 관련 질의
    if any(k in lowered for k in graduation_keywords):
        return "졸업 논문이나 졸업 프로젝트와 관련된 사항은 전공 학과나 학교의 졸업 요건에 따라 다르며, 학과 사무실이나 지도 교수님을 통해 반드시 확인하셔야 합니다."

    # 특정 학점 + 과목 타입 필터
    if re.search(r'\d\s*학점', user_input):
        return get_lectures_by_credit_and_type(user_input)

    # 팀플 조건 분석 및 응답
    team_condition = normalize_team_condition(user_input)
    match = re.search(r'(\d+)\s*개', user_input)
    count = int(match.group(1)) if match else None

    if team_condition:
        filtered_lectures = filter_lectures_by_team(["./data/kw_chatbot_data - 강의 평점.json"], team_condition)
        seen = set()
        unique_lectures = []
        for lec in filtered_lectures:
            if lec["lecture_name"] not in seen:
                seen.add(lec["lecture_name"])
                unique_lectures.append(lec)
        if count:
            unique_lectures = unique_lectures[:count]
        return format_team_project_response(team_condition, unique_lectures)

    # 성적 이하 필터
    if "이하" in normalized:
        for grade in GRADE_TO_POINT:
            if grade.lower() in normalized:
                threshold = GRADE_TO_POINT[grade]
                target_grades = [g for g, p in GRADE_TO_POINT.items() if p <= threshold]
                subjects = [r['lecture_name'] for r in load_student_data(user_id) if r['lecutre_grade'] in target_grades]
                return f"{', '.join(subjects)} 과목이 {grade} 이하입니다." if subjects else f"{grade} 이하인 과목이 없습니다."

    # 전체 성적 평균 요청
    if any(k in normalized for k in ["전체성적평균", "전체학점평균"]):
        gpas = calculate_gpa_from_raw_split(user_id)
        if "전체" in gpas:
            return f"{USER_NAME.get(user_id, user_id)}님의 전체 성적 평균은 {gpas['전체']}입니다."
        return "전체 성적 평균은 제공된 정보에 포함되어 있지 않습니다."

    # 전공 성적 평균 요청
    if "전공" in normalized and "평균" in normalized:
        gpas = calculate_gpa_from_raw_split(user_id)
        gpa = gpas.get("전공", None)
        name = USER_NAME.get(user_id, user_id)
        if gpa is not None:
            return f"{name}님의 전공 성적 평균은 {gpa}입니다."
        return f"{name}님의 전공 성적 평균은 제공된 정보에 포함되어 있지 않습니다."

    # 재수강 횟수 조회
    if "재수강몇번했어" in normalized or "나몇번재수강했어" in normalized:
        retake_counts = count_retake_courses(user_id)
        if not retake_counts:
            return "재수강한 과목이 없습니다."
        parts = [f"{name} 과목을 {count}번" for name, count in retake_counts.items()]
        return f"{', '.join(parts)} 재수강하셨습니다."

    # 단일 과목 성적 조회
    if "성적이어때" in normalized or "학점이어때" in normalized:
        course_name = user_input.split()[0].replace(" ", "")
        grade = get_course_grade(user_id, course_name)
        return f"{course_name}의 성적은 {grade}입니다." if grade else f"{course_name}에 대한 성적 정보를 찾을 수 없습니다."

    # 특정 성적 받은 과목 나열
    for grade in GRADE_TO_POINT:
        if grade.lower() in normalized and ("받은과목" in normalized or "받은수업" in normalized):
            subjects = [r['lecture_name'] for r in load_student_data(user_id) if r['lecutre_grade'] == grade]
            return f"{', '.join(subjects)} 과목에서 {grade}를 받으셨습니다." if subjects else f"{grade}를 받은 과목이 없습니다."

    # 재수강 목록 조회
    if "재수강한거뭐있어" in normalized or "재수강한과목" in normalized:
        data = load_student_data(user_id)
        names = [r['lecture_name'] for r in data if r.get('retake_status') == 'R']
        return ", ".join(f"{name}을 재수강했습니다." for name in names) if names else "재수강한 과목이 없습니다."

    # 재수강 반영한 평점 요청
    if any(k in normalized for k in ["재수강하고전체학점", "재수강반영된학점", "재수강적용학점", "재수강하고학점"]):
        return get_gpa_with_retake(user_id, exclude_pre_retake=True)

    # 재수강 가능한 전공 과목 요청
    if "재수강가능한전공과목" in normalized and any(k in normalized for k in ["뭐가있어", "뭐있어", "뭐야"]):
        filtered = get_filtered_retake_courses(user_id, focus="major")
        return f"지금 재수강 가능한 전공 과목은 {', '.join(filtered)}입니다." if filtered else "지금 재수강 가능한 전공 과목은 없습니다."

    # 전체 재수강 가능한 과목
    if "재수강가능한과목" in normalized and any(k in normalized for k in ["뭐가있어", "뭐있어", "뭐야"]):
        majors = get_filtered_retake_courses(user_id, focus="major")
        liberals = get_filtered_retake_courses(user_id, focus="liberal")
        total = majors + liberals
        return f"지금 재수강 가능한 과목은 {', '.join(total)}입니다." if total else "지금 재수강 가능한 과목은 없습니다."

    if "재수강가능한횟수" in normalized or ("재수강" in normalized and "횟수" in normalized):
        data = load_student_data(user_id)
        retake_count = sum(1 for r in data if r.get("retake_status") == "R" and r.get("lecutre_grade") != "F")
        remaining = 8 - retake_count
        name = USER_NAME.get(user_id, user_id)
        return f"{name}님은 총 8번까지 재수강이 가능하며, 현재까지 {retake_count}번 재수강하셨습니다. 따라서 {remaining}번 더 재수강할 수 있습니다."

    # 성적 향상 가정 시 GPA 시뮬레이션
    if ("재수강가능한과목" in normalized or "재수강가능한전공과목" in normalized) and "성적올리면" in normalized:
        focus = "major" if "전공" in normalized else "all"
        fixed_grade = None
        for grade in GRADE_TO_POINT:
            if f"{grade.lower()}로" in normalized:
                fixed_grade = grade
                break
        return simulate_retake_gpa(user_id, fixed_grade=fixed_grade, focus=focus)

    if "조기졸업" in normalized and "가능" in normalized:
        data = load_student_data(user_id)
        credits = sum(r['lecture_credit'] for r in data if r.get('retake_or_delete_status') != "Y")
        grades = [GRADE_TO_POINT.get(r['lecutre_grade'], 0.0) for r in data if r.get('retake_or_delete_status') != "Y"]
        avg = round(sum(grades) / len(grades), 2) if grades else 0
    
        # C+ 이하 여부 체크
        bad_grades = {"C+", "C0", "F"}
        has_c_or_lower = any(
            r.get('lecutre_grade') in bad_grades
            for r in data if r.get('retake_or_delete_status') != "Y"
        )
    
        if credits >= 114 and avg >= 4.3 and not has_c_or_lower:
            return "현재 조기 졸업 조건을 충족하셨습니다. 신청이 가능합니다."
        else:
            message = (
                "조기 졸업 가능 조건은 6학기 말까지 114학점 이상 취득(계절수업 제외), "
                "7개 학기 이상 이수자로 학칙시행세칙 제16조 평점평균의 산출에 따른 총 평점평균이 4.3 이상인 자(학적부성적 기준), "
                "조기졸업 신청 당해 학기까지 취득한 전과목 성적이 B0 이상인 자(학적부성적 기준)입니다."
            )
            if has_c_or_lower:
                message += " 현재 C+ 이하의 성적이 존재하므로 재수강을 통해 성적을 개선하고 평점 평균을 4.3 이상으로 만든다면 조기 졸업이 가능합니다."
            return message

    if "장학금" in normalized:
        if any(k in normalized for k in ["화도", "동해"]) and "최소" in normalized and "학점" in normalized:
            return "화도·동해 장학금은 직전 학기 12학점 이상, 평점 2.0 이상이어야 신청 가능합니다."
        if "성적" in normalized:
            return "성적 장학금은 평량평균이 3.0 이상이며, 17학점 이상 취득(4학년은 12학점), 그 중 60% 이상이 전공 학점일 경우 자동 신청됩니다."

    if "졸업필수과목" in normalized:
        return check_graduation_required_courses(user_id, "data/kw_chatbot_data - 수강신청자료집.json")

    if "졸업하려면몇학점남았어" in normalized:
        return get_remaining_credits(user_id)
        
    ####################
    # 단순 고정 응답 처리
    
    if "졸업유예" in normalized:
        return "아니요. 우리 학교에서는 졸업 유예 신청이 불가능합니다."

    if "재수강" in normalized and "표시" in normalized:
        return "재수강한 과목은 성적표에 ‘R’(Retake)로 표기되며, 재수강한 과목의 성적만 성적 및 학점에 반영됩니다."

    if "f받고재수강" in normalized or ("f" in normalized and "재수강" in normalized):
        return "F 받은 과목은 재수강 시 '재수강' 표시가 뜨지 않지만, 재수강 이력은 남고 성적표에는 'R'로 표기됩니다."

    if "동일과목" in normalized and "몇번" in normalized:
        return "C+ 이하인 동일과목은 최대 2회까지 재수강 가능하며, 전체 재학 중 재수강 가능한 교과목 수는 최대 8과목입니다. F 학점은 제외됩니다."
        
    if "휴학" in normalized and "몇번" in normalized:
        return "일반휴학을 최대 6학기(3년) 할 수 있으며, 한 번 신청 시 최대 2학기 가능합니다. 군휴학 및 창업휴학은 일반휴학 기간에 포함되지 않습니다. 특별한 사유가 있을 경우 추가 휴학이 가능할 수도 있습니다. (단, 일반휴학기간은 한 번에 두 학기를 초과하지 못하며, 통산 3년(6학기)(연속해서는 2년)을 초과하지 못합니다.)"

    if "같은과목" in normalized and "재수강" in normalized and ("교수" in normalized or "사라지" in normalized or "폐지" in normalized):
        return "학정번호 기준으로 동일한 학정번호가 있는 강의를 수강하시면 됩니다. 다만, 동일한 학정번호의 강의가 3년 이상 미개설 할 경우, C이하의 과목은 학점삭제 가능합니다. 자세한 내용은 학사에 문의하세요."
    ####################    
    
    #재수강 vs 새로운 전공과목 선택 질문
    if any(x in normalized for x in ["재수강", "다시듣는", "다시 들어야", "낮은성적"]) and any(y in normalized for y in ["전공", "필수", "과목", "수업"]):
        data = load_student_data(user_id)
        low_grade_threshold = 2.5
        low_major_required = []

        for record in data:
            grade_str = record.get("lecutre_grade")
            grade = GRADE_TO_POINT.get(grade_str, 5)
            ctype = record["lecture_course_type"]
            name = record["lecture_name"]

            if not ctype.startswith("전"):
                continue
            if grade > low_grade_threshold:
                continue
            if record.get("retake_or_delete_status") == "Y":
                continue

            low_major_required.append((grade_str, name))

        name = USER_NAME.get(user_id, user_id)
        response = f"재수강 여부는 개인의 학업 목표와 상황에 따라 다르지만, 성적이 낮은 전공 과목을 재수강하는 것은 전공에 대한 이해도를 높이고 평점 평균을 개선하는 데 도움이 될 수 있습니다. 반면, 다른 전공 과목을 듣는다면 다양한 지식을 쌓을 수 있는 기회가 될 수 있습니다."

        if not low_major_required:
            response += f" 현재 성적을 고려할 때, 성적이 낮은 전공 과목은 확인되지 않았습니다."
        else:
            parts = [f"{grade}를 받은 {name} 과목" for grade, name in low_major_required]
            response += " 현재 성적을 고려할 때, 재수강이 가능한 전공 과목으로는 " + " / ".join(parts) + "이 있습니다."

        return response
   
    if "이수한과목" in normalized or "이수한수업" in normalized or "과목리스트" in normalized or "수업리스트" in normalized or "들은과목" in normalized or "지금까지이수한" in normalized:
        data = load_student_data(user_id)
        name = USER_NAME.get(user_id, user_id)
        grouped = defaultdict(list)
    
        for r in data:
            if r.get("retake_or_delete_status") == "Y":
                continue
            ctype = r["lecture_course_type"]
            grade = r["lecutre_grade"]
            lecture = r["lecture_name"]
            retake_mark = ", R" if r.get("retake_status") == "R" else ""
            grouped[ctype].append(f"{lecture}({grade}{retake_mark})")
    
        order = ["전필", "전선", "교필", "교선"]
        lines = [f"{ctype}: {' / '.join(grouped[ctype])}" for ctype in order if ctype in grouped]
    
        if not lines:
            return f"{name}님께서 이수하신 과목이 확인되지 않았습니다."
        return f"{name}님께서 현재까지 이수하신 과목 목록:\n" + "\n".join(lines)

    # 2. 시나리오 매칭
    scenario_ans = match_scenario_answer(user_input)
    if scenario_ans:
        return scenario_ans

    # 3. 벡터 검색 결과에서 응답 찾기
    similar_docs = vectorstore.similarity_search_with_score(user_input, k=3)
    for doc, score in similar_docs:
        if "질문:" in doc.page_content and "답변:" in doc.page_content:
            try:
                return doc.page_content.split("답변:")[1].strip()
            except:
                continue

    # 4. LLM 기반 QA 시도
    try:
        result = qa.invoke({"query": user_input})["result"].strip()
        if result and "찾을 수 없습니다" not in result:
            return result
    except Exception:
        pass

    # 🔧 5. 최후 fallback - 세부전공 기반 시나리오 중 가장 유사한 항목
    return find_best_matching_answer(user_input, user_id)

def find_best_matching_answer(user_input, user_id):
    major = "VT" if user_id == "kim" else "DS"
    major_data = [item for item in SCENARIOS if item.get("세부전공") == major]
    if not major_data:
        major_data = SCENARIOS  # fallback to all if major tag missing

    norm_input = user_input.replace(" ", "").lower()
    best_score = 0
    best_answer = None

    for item in major_data:
        q = item.get("Text", "")
        a = item.get("Completion", "")
        q_norm = q.replace(" ", "").lower()
        score = SequenceMatcher(None, norm_input, q_norm).ratio()
        if score > best_score:
            best_score = score
            best_answer = a

    return best_answer if best_answer else "해당 질문에 대한 적절한 답변을 찾을 수 없습니다."

In [53]:
import json
import difflib

# JSON 로딩 (최초 1회만)
with open("./data/진로상담.json", encoding="utf-8") as f:
    counsel_data = json.load(f)

# 정확 매칭 + 유사도 매칭 기반 질의 처리기
def find_best_matching_answer(user_input, user_id):
    major = "VT" if user_id == "kim" else "DS"
    major_data = [item for item in counsel_data if item["세부전공"] == major]

    # 1. 완전 일치 먼저 시도
    for item in major_data:
        if item["Text"].strip() == user_input.strip():
            return item["Completion"]

    # 2. 유사도 높은 질문 탐색
    question_list = [item["Text"] for item in major_data]
    close = difflib.get_close_matches(user_input, question_list, n=1, cutoff=0.65)
    if close:
        for item in major_data:
            if item["Text"] == close[0]:
                return item["Completion"]

    return "현재 전공에서는 해당 질문에 대한 답변을 찾을 수 없습니다."

# 메인 응답 함수
def answer_query(user_input, user_id="kim"):
    return find_best_matching_answer(user_input, user_id)

#### 수강이력

##### 학점 계산

In [56]:
answer_query("전체 성적 평균이 어떻게 돼?", user_id="hong")

KeyError: '세부전공'

In [38]:
answer_query("전체 성적 평균이 어떻게 돼?", user_id="kim")

'현재 전공에서는 해당 질문에 대한 답변을 찾을 수 없습니다.'

In [52]:
answer_query("전공 평균이 어떻게 돼?", user_id="hong")

'홍데사님의 전공 성적 평균은 3.45입니다.'

In [49]:
answer_query("전공 성적 평균이 어떻게 돼?", user_id="kim")

'김브티님의 전공 성적 평균은 3.46입니다.'

In [210]:
answer_query("지금 재수강가능한 과목들을 A0로 성적올리면 전체 학점이 어떻게 돼?", user_id="kim")

'AI수학: A0 / 정보디자인프로그래밍: A0 → 예상 평점: 3.85'

In [209]:
answer_query("지금 재수강가능한 과목들을 B0로 성적올리면 전체 학점이 어떻게 돼?", user_id="kim")

'AI수학: B0 / 정보디자인프로그래밍: B0 → 예상 평점: 3.76'

In [214]:
answer_query("지금 재수강가능한 과목들을 A0로 성적올리면 전체 학점이 어떻게 돼?", user_id="hong")

'프로그래밍기초: A0 / 객체지향프로그래밍: A0 / 정보디자인프로그래밍: A0 → 예상 평점: 3.89'

##### 재수강

In [18]:
answer_query("재수강 몇 번 했어?", user_id="hong")

'창의설계입문 과목을 1번 재수강하셨습니다.'

In [19]:
answer_query("나 몇 번 재수강했어?", user_id="kim")

'프로그래밍기초 과목을 1번 재수강하셨습니다.'

In [95]:
answer_query("재수강한 과목이 뭐야?", user_id="kim")

'프로그래밍기초을 재수강했습니다.'

In [21]:
answer_query("재수강한 거 뭐있어?", user_id="hong")

'창의설계입문을 재수강했습니다.'

In [77]:
answer_query("지금 재수강가능한 과목은 뭐야?", user_id="hong")

'지금 재수강 가능한 과목은 프로그래밍기초, 객체지향프로그래밍, 정보디자인프로그래밍입니다.'

In [26]:
answer_query("지금 재수강가능한 전공과목은 뭐야?", user_id="hong")

'지금 재수강 가능한 전공 과목은 없습니다.'

In [27]:
answer_query("지금 재수강가능한 과목은 뭐야?", user_id="kim")

'지금 재수강 가능한 과목은 AI수학, 정보디자인프로그래밍입니다.'

In [28]:
answer_query("지금 재수강가능한 전공과목은 뭐야?", user_id="kim")

'지금 재수강 가능한 전공 과목은 없습니다.'

##### 과목 질문

In [29]:
answer_query("내가 A+ 받은 과목 알려줘", user_id="kim")

'경제와경영, 대학수학및연습2, 인터넷활용, 생활속의생명과학 과목에서 A+를 받으셨습니다.'

In [30]:
answer_query("내가 A+ 받은 과목 알려줘", user_id="hong")

'컴퓨팅사고, 대학수학및연습1, 그래픽디자인, 대학수학및연습2, AI수학, 화폐와금융의과거,현재그리고미래, 생활속의생명과학 과목에서 A+를 받으셨습니다.'

In [31]:
answer_query("고급C프로그래밍 학점이 어때?", user_id="kim")

'고급C프로그래밍의 성적은 B+입니다.'

In [32]:
answer_query("객체지향프로그래밍 성적이 뭐야?", user_id="kim")

'객체지향프로그래밍의 성적은 B0입니다.'

In [33]:
answer_query("C+ 이하 과목만 보고 싶어", user_id="hong")

'창의설계입문, 프로그래밍기초, 객체지향프로그래밍, 정보디자인프로그래밍 과목이 C+ 이하입니다.'

#### 강의탐색

In [222]:
print(answer_query("팀플이 없는 강의 알려줘"))

팀플이 없는 강의는 다음과 같습니다:
1. 컴퓨터네트워크 (학정번호: I040-2-1655-01)
2. AI수학 (학정번호: I040-2-3688-01)
3. 객체지향프로그래밍 (학정번호: I040-2-7777-01)
4. 오픈소스소프트웨어 (학정번호: I040-2-8461-01)
5. 인터랙티브미디어개론 (학정번호: I040-2-9160-01)
6. 컴퓨터그래픽스 (학정번호: I040-3-3951-01)
7. 컴퓨터비전 (학정번호: I040-3-5474-01)
8. 실험설계및분석 (학정번호: I040-3-9617-01)
9. 영상AI생성모델 (학정번호: I040-4-5471-01)


In [216]:
print(answer_query("팀플 없는 강의 알려줘"))

팀플이 없는 강의는 다음과 같습니다:
1. 컴퓨터네트워크 (학정번호: I040-2-1655-01)
2. AI수학 (학정번호: I040-2-3688-01)
3. 객체지향프로그래밍 (학정번호: I040-2-7777-01)
4. 오픈소스소프트웨어 (학정번호: I040-2-8461-01)
5. 인터랙티브미디어개론 (학정번호: I040-2-9160-01)
6. 컴퓨터그래픽스 (학정번호: I040-3-3951-01)
7. 컴퓨터비전 (학정번호: I040-3-5474-01)
8. 실험설계및분석 (학정번호: I040-3-9617-01)
9. 영상AI생성모델 (학정번호: I040-4-5471-01)


In [224]:
print(answer_query("조별과제가 적은 강의 알려줘"))

팀플이 적은 강의는 다음과 같습니다:
1. 빅데이터프로그래밍 (학정번호: I040-2-4501-01)
2. 텍스트마이닝 (학정번호: I040-3-4139-01)
3. 딥러닝프로그래밍 (학정번호: I040-4-4434-01)
4. 산학협력캡스톤설계1 (학정번호: I040-4-8995-01)
5. 데이터시각화 (학정번호: I040-4-9925-01)


In [225]:
print(answer_query("조별과제 적은 강의 알려줘"))

팀플이 적은 강의는 다음과 같습니다:
1. 빅데이터프로그래밍 (학정번호: I040-2-4501-01)
2. 텍스트마이닝 (학정번호: I040-3-4139-01)
3. 딥러닝프로그래밍 (학정번호: I040-4-4434-01)
4. 산학협력캡스톤설계1 (학정번호: I040-4-8995-01)
5. 데이터시각화 (학정번호: I040-4-9925-01)


In [226]:
print(answer_query("조별과제 많은 강의 알려줘"))

팀플이 많은 강의는 다음과 같습니다:
1. IoT시스템설계및실습 (학정번호: I040-3-4503-01)
2. UX/UI디자인 (학정번호: I040-3-7737-01)
3. 기계학습 (학정번호: I040-3-9151-01)


In [221]:
print(answer_query("조별과제가 많은 강의 알려줘"))

팀플이 많은 강의는 다음과 같습니다:
1. IoT시스템설계및실습 (학정번호: I040-3-4503-01)
2. UX/UI디자인 (학정번호: I040-3-7737-01)
3. 기계학습 (학정번호: I040-3-9151-01)


In [45]:
print(answer_query("전선 중 3학점 5개만 추천해줘"))

3학점짜리 전선 과목 목록입니다:
1. 데이터시각화 (학정번호: I040-4-9925-01) - 조재희
2. 인체데이터분석및실습 (학정번호: I000-2-5715-01) - 고찬영
3. 공학설계입문 (학정번호: I000-1-2957-02) - 임재한
4. 제어알고리즘설계및실습 (학정번호: I000-2-5702-01) - 고찬영
5. 디지털헬스케어인공지능분석실무 (학정번호: I020-2-4888-01) - 교수명 없음


In [92]:
print(answer_query("전공 중 3학점 5개만 추천해줘"))

3학점짜리 전필, 전선 과목 목록입니다:
1. 딥러닝프로그래밍 (학정번호: I040-4-4434-01) - 김준석
2. 성공적인창업계획 (학정번호: 5080-4-7882-01) - 이현경
3. 컴퓨터그래픽스 (학정번호: I040-3-3951-01) - 김동준
4. 영상AI생성모델 (학정번호: I040-4-5471-01) - 김동준
5. 창의설계입문 (학정번호: I000-1-3674-01) - 신원경


In [47]:
print(answer_query("교양 중 3학점 5개만 추천해줘"))

3학점짜리 교필, 교선 과목 목록입니다:
1. 실용중국어문법 (학정번호: 0000-2-6526-01) - 곡효운
2. 평생교육론 (학정번호: 0000-1-7468-01) - 김성길
3. C프로그래밍 (학정번호: 0000-1-0019-09) - 박상준
4. 대학수학및연습1 (학정번호: 0000-1-4625-16) - 김원주
5. 한국가족의역사와현재 (학정번호: 0000-2-4833-01) - 김인호


In [93]:
print(answer_query("교양 중 2학점 5개만 추천해줘"))

2학점짜리 교필, 교선 과목 목록입니다:
1. 탁구 (학정번호: 0000-1-1679-01) - 박현애
2. 스케이팅 (학정번호: 0000-1-2238-01) - 정재은
3. 웰니스트레이닝 (학정번호: 0000-1-5684-02) - 박인혜
4. 사회봉사1 (학정번호: 0000-1-3902-01) - 임안나
5. 저작권과스마트폰의이해 (학정번호: 0000-1-8129-01) - 건국대학교 정연덕


In [219]:
print(answer_query("3학년 1학기에 들을 만한 과목 추천해줘."))

3학년 1학기에 들을 만한 과목으로는 "고급C프로그래밍", "자료구조", "컴퓨터네트워크", "모바일프로그래밍"이 있습니다. 이 과목들은 전공 필수 및 선택 과목으로, 정보융합학부에서 학습하는 데 도움이 될 것입니다.


#### 학업현황

###### 졸업

In [118]:
answer_query("졸업 필수 과목을 다 들었어?", user_id="hong")

'교필 과목인 광운인되기를 이수하지 않았습니다.'

In [117]:
answer_query("졸업하려면 몇 학점 남았어?", user_id="hong")

'홍데사님의 현재까지 이수한 학점은 69학점이며, 졸업을 위해 64학점이 더 필요합니다.'

In [156]:
answer_query("졸업 논문이나 졸업 프로젝트가 필수야??", user_id="hong")

'졸업하기 위해서는 졸업 논문 혹은 졸업 프로젝트 둘 중 하나는 필수입니다.'

In [155]:
answer_query("졸업 논문 필수야??", user_id="hong")

'졸업 논문은 필수입니다.'

In [154]:
answer_query("졸업 논문이나 프로젝트는 언제부터 준비해야 할까?", user_id="hong")

'빠르면 3학년부터 준비하는 경우도 있지만, 대부분 3학년 2학기부터 팀을 구한 후 4학년 때 프로젝트를 진행합니다. 논문의 경우도 대부분 4학년 때 준비합니다.'

In [79]:
answer_query("졸업까지 최소 몇 학기 남았어?", user_id="kim")

'졸업까지 최소 4학기가 남았습니다.'

In [149]:
answer_query("우리 학교에서 졸업 유예 신청이 가능해?", user_id="hong")

'아니요. 우리 학교에서는 졸업 유예 신청이 불가능합니다.'

In [152]:
answer_query("조기 졸업이 가능한 상황이야?", user_id="hong")

'조기 졸업 가능 조건은 6학기 말까지 114학점 이상 취득(계절수업 제외), 7개 학기 이상 이수자로 학칙시행세칙 제16조 평점평균의 산출에 따른 총 평점평균이 4.3 이상인 자(학적부성적 기준), 조기졸업 신청 당해 학기까지 취득한 전과목 성적이 B0 이상인 자(학적부성적 기준)입니다. 현재 C+ 이하의 성적이 존재하므로 재수강을 통해 성적을 개선하고 평점 평균을 4.3 이상으로 만든다면 조기 졸업이 가능합니다.'

##### 재수강

In [131]:
answer_query("지금 재수강가능한 과목들을 A0로 성적올리면 전체 학점이 어떻게 돼?", user_id="kim")

'AI수학: A0 / 정보디자인프로그래밍: A0 → 예상 평점: 3.85'

In [151]:
answer_query("지금 재수강가능한 과목은 뭐야?", user_id="hong")

'지금 재수강 가능한 과목은 프로그래밍기초, 객체지향프로그래밍, 정보디자인프로그래밍입니다.'

In [176]:
answer_query("재수강 가능한 횟수가 어떻게 돼?", user_id="hong")

'홍데사님은 총 8번까지 재수강이 가능하며, 현재까지 1번 재수강하셨습니다. 따라서 7번 더 재수강할 수 있습니다.'

In [177]:
answer_query("재수강 가능한 횟수가 어떻게 돼?", user_id="kim")

'김브티님은 총 8번까지 재수강이 가능하며, 현재까지 1번 재수강하셨습니다. 따라서 7번 더 재수강할 수 있습니다.'

In [178]:
answer_query("내가 재수강한 과목이 뭔데?", user_id="kim")

'프로그래밍기초을 재수강했습니다.'

In [171]:
answer_query("같은 과목을 재수강할 때 교수님이 바뀌거나 과목이 사라지면 어떻게 해야 해?", user_id="hong")

'학정번호 기준으로 동일한 학정번호가 있는 강의를 수강하시면 됩니다. 다만, 동일한 학정번호의 강의가 3년 이상 미개설 할 경우, C이하의 과목은 학점삭제 가능합니다. 자세한 내용은 학사에 문의하세요.'

In [175]:
answer_query("재수강하면 성적표에 재수강 여부가 표시되나?", user_id="hong")

'재수강한 과목은 성적표에 ‘R’(Retake)로 표기되며, 재수강한 과목의 성적만 성적 및 학점에 반영됩니다.'

In [174]:
answer_query("F받고 재수강하면 재수강 이력이 남아?", user_id="hong")

"F 받은 과목은 재수강 시 '재수강' 표시가 뜨지 않지만, 재수강 이력은 남고 성적표에는 'R'로 표기됩니다."

In [172]:
answer_query("성적이 낮은 전공 필수 과목을 재수강하는 게 나을까, 다른 전공 과목을 듣는 게 나을까?", user_id="hong")

'재수강 여부는 개인의 학업 목표와 상황에 따라 다르지만, 성적이 낮은 전공 과목을 재수강하는 것은 전공에 대한 이해도를 높이고 평점 평균을 개선하는 데 도움이 될 수 있습니다. 반면, 다른 전공 과목을 듣는다면 다양한 지식을 쌓을 수 있는 기회가 될 수 있습니다. 현재 성적을 고려할 때, 재수강이 가능한 전공 과목으로는 C+를 받은 객체지향프로그래밍 과목 / C+를 받은 정보디자인프로그래밍 과목이 있습니다.'

In [173]:
answer_query("동일 과목은 재수강 몇 번 할 수 있어?", user_id="hong")

'C+ 이하인 동일과목은 최대 2회까지 재수강 가능하며, 전체 재학 중 재수강 가능한 교과목 수는 최대 8과목입니다. F 학점은 제외됩니다.'

##### 학업현황

In [87]:
print(answer_query("지금까지 이수한 과목 리스트를 볼 수 있을까?", user_id="hong"))

홍데사님께서 현재까지 이수하신 과목 목록:
전필: 객체지향프로그래밍(C+) / AI수학(A+) / 자료구조(B+) / 모바일프로그래밍(B+)
전선: 고급C프로그래밍(B+) / 이산수학(B0) / 그래픽디자인(A+) / 창의설계입문(A0, R) / 빅데이터프로그래밍(B0) / 데이터베이스(B+) / 정보디자인프로그래밍(C+)
교필: 프로그래밍기초(C+) / 융합적사고와글쓰기(B+) / 컴퓨팅사고(A+)
교선: 대학수학및연습1(A+) / 스케이팅(B+) / 대학수학및연습2(A+) / 아카펠라(A0) / 범죄학(B+) / 대중문화와삶(B+) / 화폐와금융의과거,현재그리고미래(A+) / 사회학의이해(A0) / 생활속의생명과학(A+)


In [179]:
print(answer_query("지금까지 이수한 과목 리스트를 볼 수 있을까?", user_id="kim"))

김브티님께서 현재까지 이수하신 과목 목록:
전필: 객체지향프로그래밍(B0) / AI수학(C+) / 자료구조(B+) / 모바일프로그래밍(A0)
전선: 창의설계입문(A0) / 고급C프로그래밍(B+) / 이산수학(A0) / 그래픽디자인(B+) / 컴퓨터네트워크(A0) / 인터랙티브미디어개론(B0) / 인터랙티브심리학(A0) / 정보디자인프로그래밍(C+)
교필: 프로그래밍기초(A0, R)
교선: 대학수학및연습1(B+) / 초급중국어1(A0) / 경제와경영(A+) / 자연과학사(B+) / 대학수학및연습2(A+) / 환경과생태(B+) / 논리적으로사고하기(B+) / 인터넷활용(A+) / 사회학의이해(A0) / 생활속의생명과학(A+)


In [78]:
answer_query("전체 성적 평균이 어떻게 돼?", user_id="hong")

'홍데사님의 전체 성적 평균은 3.7입니다.'

##### 장학금&휴학

In [186]:
answer_query("성적 장학금을 받기 위한 조건이 뭐야?", user_id="hong")

'성적 장학금은 평량평균이 3.0 이상이며, 17학점 이상 취득(4학년은 12학점), 그 중 60% 이상이 전공 학점일 경우 자동 신청됩니다.'

In [185]:
answer_query("화도 동해 장학금 최소 학점이 얼마야? ", user_id="hong")

'화도·동해 장학금은 직전 학기 12학점 이상, 평점 2.0 이상이어야 신청 가능합니다.'

In [184]:
answer_query("휴학은 최대 몇 번까지 가능해?", user_id="hong")

'일반휴학을 최대 6학기(3년) 할 수 있으며, 한 번 신청 시 최대 2학기 가능합니다. 군휴학 및 창업휴학은 일반휴학 기간에 포함되지 않습니다. 특별한 사유가 있을 경우 추가 휴학이 가능할 수도 있습니다. (단, 일반휴학기간은 한 번에 두 학기를 초과하지 못하며, 통산 3년(6학기)(연속해서는 2년)을 초과하지 못합니다.)'

#### 진로상담

##### 김브티

###### 진로에 따른 강의 추천

In [238]:
answer_query("처음 프론트 분야를 배우는데 어떤 과목을 들어야 해?", user_id="kim")

'처음 프론트엔드 분야를 배우려면 1학기에는 인터랙티브 미디어 개론 수업을 듣는 것이 좋습니다. 이 수업에서는 HTML과 CSS 등을 활용해 홈페이지를 만들면서 디자인 경험을 쌓을 수 있습니다. 이후 2학기에는 정보디자인프로그래밍 수업을 들으면서 팀 프로젝트를 통해 프론트엔드 개발을 실습해볼 수 있습니다. 이를 통해 프론트엔드 직무를 간접적으로 경험해볼 수 있습니다.'

In [258]:
answer_query("처음 프론트 분야를 해보면 어떤 과목이 좋을까?", user_id="kim")

'처음 프론트엔드 분야를 배우려면 1학기에는 인터랙티브 미디어 개론 수업을 듣는 것이 좋습니다. 이 수업에서는 HTML과 CSS 등을 활용해 홈페이지를 만들면서 디자인 경험을 쌓을 수 있습니다. 이후 2학기에는 정보디자인프로그래밍 수업을 들으면서 팀 프로젝트를 통해 프론트엔드 개발을 실습해볼 수 있습니다. 이를 통해 프론트엔드 직무를 간접적으로 경험해볼 수 있습니다.'

In [239]:
answer_query("프론트 실습이 많은 수업 추천해줘.", user_id="kim")

'2학년 1학기에 열리는 인터랙티브 미디어 개론 수업과 2학년 2학기에 열리는 정보디자인프로그래밍 수업을 추천합니다.'

In [259]:
answer_query("프론트 실습이 많은 수업 뭐 있어?", user_id="kim")

'2학년 1학기에 열리는 인터랙티브 미디어 개론 수업과 2학년 2학기에 열리는 정보디자인프로그래밍 수업을 추천합니다.'

In [240]:
answer_query("데이터 사이언스 및 AI 직무를 준비하려면 꼭 들어야 할 과목이 뭐야?", user_id="kim")

'빅데이터 프로그래밍, 데이터베이스, AI수학, 기계학습, 텍스트마이닝, 컴퓨터비전, 데이터마이닝, 데이터사이언스개론 수업등을 추천합니다.'

In [241]:
answer_query("학교 수업들 중에 협업 경험을 쌓을 수 있게 팀프로젝트를 진행하는 수업 추천해줘", user_id="kim")

'기계학습, 데이터마이닝, 웹서비스설계및실습, 정보디자인프로그래밍, 텍스트마이닝 등 수업에서 팀프로젝트를 경험해 볼 수 있습니다.'

In [242]:
answer_query("클라우드(AWS, GCP 등) 기술을 다루는 수업이 있나?", user_id="kim")

'오픈소스소프트웨어 수업과 웹서비스설계및실습 수업에서 경험해 볼 수 있습니다.'

###### 진로상담

In [243]:
answer_query("자대 대학원 진학시 이점이 궁금해", user_id="kim")

'기존에 알고 있는 교수님이나 연구실 네트워크를 활용하기 쉬워 안정적인 연구 환경을 만들 수 있습니다. 행정적·학문적 연속성도 이어갈 수 있어 적응 기간이 짧습니다.'

In [244]:
answer_query("현재 VT 전공을 하고 있는데 직무를 못 정하고 있는데 어떻게 해야 할까?", user_id="kim")

'관심 있는 분야를 중심으로 짧은 실습이나 프로젝트를 먼저 해보시길 추천드립니다. 실제 경험이 직무 선택에 큰 도움을 줍니다.'

In [245]:
answer_query("UI/UX 디자인에 관심 많은데 어떤 직업이 적합할까?", user_id="kim")

'UX 디자이너, UI 디자이너, 프로덕트 디자이너 같은 직무가 잘 맞습니다. 사용자 중심 사고와 디자인 툴(Figma 등) 역량이 중요합니다.'

In [246]:
answer_query("석사 진학시 이후 진로가 궁금해~", user_id="kim")

'석사 이후에는 R&D 연구원, 데이터사이언티스트, AI 개발자 등 전문 직무로 나아갈 수 있습니다. 박사 진학이나 산학 협력 창업도 가능성 있습니다.'

In [247]:
answer_query("만약 경험을 쌓다가 프론트 개발자가 잘 안 맞는다면 추천하는 비슷한 직무가 있어?", user_id="kim")

'UX 기획자, 웹 퍼블리셔, PM 등 시각 중심 협업 직무가 적합할 수 있습니다.'

In [248]:
answer_query("UI 디자인이랑 프론트 개발 중 어떤 진로가 나한테 맞을까?", user_id="kim")

'시각적인 표현과 구성에 흥미가 많고 툴(Figma 등) 다루는 걸 좋아하신다면 UI 디자인이 더 잘 맞을 수 있습니다. 반면 로직 구현이나 사용자 인터랙션을 직접 코딩하고 싶은 성향이면 프론트 개발 쪽이 더 적합합니다.'

###### 정보제공

In [249]:
answer_query("실무에서 많이 쓰이는 프론트 프레임워크는 뭐가 있어?", user_id="kim")

'React, Vue, Next.js가 가장 많이 사용됩니다.'

In [250]:
answer_query("UI/UX 디자인을 배우려면 기초적으로 어떤 개념부터 공부해야 해?", user_id="kim")

'UI/UX 디자인을 배우려면 먼저 사용자 중심 사고, 디자인 원칙(가독성, 정렬, 일관성 등), 그리고 정보 구조화(IA) 개념부터 이해하는 것이 중요합니다.'

In [251]:
answer_query("프론트 개발자로 성장하려면 어떤 언어 부터 시작해야 할까?", user_id="kim")

'HTML, CSS, JavaScript를 먼저 학습하고 React까지 확장하시는 걸 추천드립니다.'

In [252]:
answer_query("프론트 직무를 희망할 때 갖추면 좋은 자격증에는 어떤 것들이 있을까?", user_id="kim")

'정보처리기사, 웹디자인 기능사 등 기초 역량을 증명하는 자격증이 유용합니다.'

In [253]:
answer_query("휴학 중에 프론트 개발 역량을 키우려면 어떤 활동이 효과적이야?", user_id="kim")

'개인 웹사이트나 포트폴리오 페이지를 직접 만들고 배포해보는 것이 좋습니다. 실무와 유사한 UI 구성, 반응형 설계 경험이 도움이 됩니다.'

In [254]:
answer_query("자대 대학원 진학 예정인데 유용한 학교 연계 프로그램이 있을까?", user_id="kim")

'산학협력 연구, 기업 인턴십, 캡스톤 디자인, 기술 창업 프로그램 등 다양한 연계 활동이 있을 수 있습니다. 소속 연구실을 통해 참여 기회를 넓혀가세요.'

In [257]:
answer_query("프론트 직무에서 중요하게 평가하는 역량은 뭐야?", user_id="kim")

'UI/UX 이해, 협업 능력, 유지보수 용이한 코드 작성 역량이 중요합니다. 사용자 친화적인 화면 구성과 커뮤니케이션 능력이 함께 평가됩니다.'

In [256]:
answer_query("추천하는 프론트 관련 프로젝트 주제가 있을까?", user_id="kim")

'반응형 일정 앱, 날씨 기반 서비스 제작이 좋습니다. 실제 사용자 경험을 고려해 인터랙션을 설계해보는 것이 좋습니다.'

In [255]:
answer_query("UI 디자인이나 프론트엔드 개발 관련 직무는 어떤 것들이 있어?", user_id="kim")

'UI 디자인이나 프론트엔드 개발 관련 직무에는 UI 디자이너, 프론트엔드 개발자, 웹 퍼블리셔, 프로덕트 디자이너, 인터랙션 디자이너 등이 있습니다.'

##### 홍데사

###### 진로에 따른 강의 추천

In [260]:
answer_query("처음 백 분야를 배우는데 어떤 과목을 들어야 해?", user_id="hong")

'처음 백엔드 분야를 배우려면 1학기에는 객체지향프로그래밍 수업을 들으며 Java 언어를 학습하고, Python 외의 기본 프로그래밍 언어를 익히면서 코딩의 기초를 다질 수 있습니다.이후 2학기에는 데이터베이스 수업을 들으며 SQL과 같은 데이터 관리 기술을 학습할 수 있습니다. 그리고 3학년 2학기에는 웹 서비스 설계 및 실습 수업을 들으며 협업 프로젝트에서 백엔드 개발을 경험해볼 수 있습니다. 추가로, 백엔드 개발을 깊이 있게 공부하려면 자료구조 수업을 통해 효율적인 데이터 처리 방식을 익히고, 컴퓨터 네트워크 수업을 통해 서버와 클라이언트 간의 데이터 송수신 원리를 이해하는 것이 도움이 됩니다.'

###### 진로상담

In [264]:
answer_query("현재 DS 전공을 하고 있는데 직무를 못 정하고 있는데 어떻게 해야 할까?", user_id="hong")

'관심 있는 분야를 중심으로 짧은 실습이나 프로젝트를 먼저 해보시길 추천드립니다. 실제 경험이 직무 선택에 큰 도움을 줍니다.'

###### 정보제공

In [262]:
answer_query("실무에서 많이 쓰이는 백 프레임워크는 뭐가 있어?", user_id="hong")

'Spring Boot(Java), Django(Python), Express(Node.js)가 대표적입니다.'

#### 유사도 검사

###### 학업현황

In [68]:
from langchain_community.vectorstores import Chroma

user_input = "졸업 필수 과목을 다 들었어?"

# Chroma DB 연결
db = Chroma(
    persist_directory="./chroma_db",
    collection_name="lecture_search",
    embedding_function=embeddings_model
)


response = answer_query(user_input, user_id="hong")
print("🟢 챗봇 답변:")
print(response)

similar_docs = db.similarity_search_with_score(user_input, k=3)

print("\n📚 유사 문서:")
for i, (doc, score) in enumerate(similar_docs, 1):
    print(f"\n{i}. 유사도: {score:.4f}")
    print(doc.page_content[:300], "...")

🟢 챗봇 답변:
교필 과목인 광운인되기를 이수하지 않았습니다.

📚 유사 문서:

1. 유사도: 67.3077
- 33 -
※ 지정과목은 반드시 졸업할 때까지 이수하여야 하며, 한 과목이라도 이수하지 않을 경우에는 졸업이 불가함. 라. 3학년 1학기 편입한 학생은 4학기 이상을 등록을 하여야 하고, 2학년 1학기에 편입한 학생은 6학기 이상을 등록하여야 함. 3학년 2학기 편입생은 편입학년의 1학기에 개설된 전공필수과목을 포함하여 모두 이수하여야 함       (예: 3학년 2학기 편입생 - 졸업 이전까지 3학년 1학기 전공필수과목을 필히 이수하여야 함).마. 편입생은 교양필수 이수를 면제하고, 교양학점 30학점을 이수하도록 함.6
다전공( ...

2. 유사도: 95.0026
편입생이 이수해야 할 교과목 및 학점
가. 편입생은 수강신청을 하기 전에 반드시 소속 학과(부)장과 면담하여 전적교에서 취득한 학점을 인정받기 위한 절차를 거쳐 인정받은 학점(교양․ 전공․ 일선)과 향후 반드시 이수해야 할 교과목을 확인하고 수강 신청하여야 함.   ※ 해당 학과에서 학점 심의를 통해 학점을 인정하며, 인정된 학점 및 이수요망 교과목에 대하여는 추후 공지를 통하여 배부되며, 학교홈페이지 KLAS>수강성적조회 메뉴에서도 확인 가능. 나. 전적교에서 이수한 교과목과 동일한 과목을 본교에서 다시 이수하여 학점을 취득할 수 ...

3. 유사도: 97.3878
- 47 -
(2020학번 입학자부터 적용) 졸업요건졸업이수학점 : 133학점(필수 포함)구분학과교양 전공졸업요건 학점최소 졸업요건 ‘공학필수’ 전자정보공과대학
전자공학과
① 학번에 따른 「교양 교과목 이수체계」참고② MSC 30학점- 공학계열 단과대학 학과별 「기초교양 및 MSC(수학,기초과학,전산학)교과과정표」참고
전공 전필포함60학점(설계 12학점 포함) ...


In [67]:
from langchain_community.vectorstores import Chroma

user_input = "졸업하려면 몇 학점 남았어?"

# Chroma DB 연결
db = Chroma(
    persist_directory="./chroma_db",
    collection_name="lecture_search",
    embedding_function=embeddings_model
)


response = answer_query(user_input, user_id="hong")
print("🟢 챗봇 답변:")
print(response)

similar_docs = db.similarity_search_with_score(user_input, k=3)

print("\n📚 유사 문서:")
for i, (doc, score) in enumerate(similar_docs, 1):
    print(f"\n{i}. 유사도: {score:.4f}")
    print(doc.page_content[:300], "...")

🟢 챗봇 답변:
홍데사님의 현재까지 이수한 학점은 69학점이며, 졸업을 위해 64학점이 더 필요합니다.

📚 유사 문서:

1. 유사도: 90.8333
- 33 -
※ 지정과목은 반드시 졸업할 때까지 이수하여야 하며, 한 과목이라도 이수하지 않을 경우에는 졸업이 불가함. 라. 3학년 1학기 편입한 학생은 4학기 이상을 등록을 하여야 하고, 2학년 1학기에 편입한 학생은 6학기 이상을 등록하여야 함. 3학년 2학기 편입생은 편입학년의 1학기에 개설된 전공필수과목을 포함하여 모두 이수하여야 함       (예: 3학년 2학기 편입생 - 졸업 이전까지 3학년 1학기 전공필수과목을 필히 이수하여야 함).마. 편입생은 교양필수 이수를 면제하고, 교양학점 30학점을 이수하도록 함.6
다전공( ...

2. 유사도: 98.9711
수강신청 학점범위
가. 수강신청 학점 범위① 2016학번까지 : (최소)12학점 ~ (최대)21학점② 2017학번부터 : (최소)12학점 ~ (최대)19학점※ 졸업예정학기 (8학기, 건축학과 10학기)학생은 최소 수강신청 학점을 6학점으로 적용함.※ 8학기 (건축학과 10학기) 초과 수강신청자는 최저학점 (12학점) 미만으로 수강신청 가능함.※ 다전공, 재수강, 타 대학 학점교류 등의 학점은 최대 수강신청학점에 포함됨.※ 수강신청 학점이 미달(12학점 미만, 졸업예정학기 학생 6학점 미만)되지 않도록 유의할 것.③ 학점이월제도(20 ...

3. 유사도: 101.6380
- 28 -
바. 2016학년도 신입학자  1) 졸업이수학점(2017학년도 2학년 편입생, 2018학년도 3학년 편입생 포함)
   
단 과 대 학 / 학 과 교양전공(필수포함)졸업이수학점필수+균형기초전정공대, 공과대13~1624~3060140건축학과(5년제) 13~16-120170자연대수학과7~10-54140전자바이오물리학과,화학과, 생활체육학과13~16-60140정보콘텐츠학과(사이버정보보안학과)30-45120인문사회과학대학국어국문학과16~19-51140영어영문학과45140산업

In [63]:
from langchain_community.vectorstores import Chroma

user_input = "졸업 논문이나 졸업 프로젝트가 필수야?"

# Chroma DB 연결
db = Chroma(
    persist_directory="./chroma_db",
    collection_name="lecture_search",
    embedding_function=embeddings_model
)


response = answer_query(user_input, user_id="kim")
print("🟢 챗봇 답변:")
print(response)

similar_docs = db.similarity_search_with_score(user_input, k=3)

print("\n📚 유사 문서:")
for i, (doc, score) in enumerate(similar_docs, 1):
    print(f"\n{i}. 유사도: {score:.4f}")
    print(doc.page_content[:300], "...")

🟢 챗봇 답변:
졸업하기 위해서는 졸업 논문 혹은 졸업 프로젝트 둘 중 하나는 필수입니다.

📚 유사 문서:

1. 유사도: 86.9059
학∙석사연계과정 관련 문의   학부 성적 및 졸업요건 관련 문의 ...

2. 유사도: 88.0347
- 96 -
 다. 인증 요건§선택 언어별 모든 레벨에서 최소 1과목(3학점) 이상 이수(영어는 1~4레벨)§총 5과목 수강(15학점), 수강과목 평점 3.5 이상 라. 시행시기§2020년 후기졸업자부터 인증서 발급 가능 마. 혜   택 §총장 명의 인증서 수여 바. 신청 및 수령 방법§매 학기말 홈페이지 공지사항 참고§참빛인재양성 통합관리 시스템 사이트 내 신청(성적증명서 첨부)§학위수여식(2월, 8월) 이후 인제니움대학 교학팀에서 수령 가능 ...

3. 유사도: 94.0891
- 28 -
바. 2016학년도 신입학자  1) 졸업이수학점(2017학년도 2학년 편입생, 2018학년도 3학년 편입생 포함)
   
단 과 대 학 / 학 과 교양전공(필수포함)졸업이수학점필수+균형기초전정공대, 공과대13~1624~3060140건축학과(5년제) 13~16-120170자연대수학과7~10-54140전자바이오물리학과,화학과, 생활체육학과13~16-60140정보콘텐츠학과(사이버정보보안학과)30-45120인문사회과학대학국어국문학과16~19-51140영어영문학과45140산업심리학과51140미디어영상학부45140동북아문화산업학부 ...


In [64]:
from langchain_community.vectorstores import Chroma

user_input = "졸업 논문이나 프로젝트는 언제부터 준비해야 할까?"

# Chroma DB 연결
db = Chroma(
    persist_directory="./chroma_db",
    collection_name="lecture_search",
    embedding_function=embeddings_model
)


response = answer_query(user_input, user_id="kim")
print("🟢 챗봇 답변:")
print(response)

similar_docs = db.similarity_search_with_score(user_input, k=3)

print("\n📚 유사 문서:")
for i, (doc, score) in enumerate(similar_docs, 1):
    print(f"\n{i}. 유사도: {score:.4f}")
    print(doc.page_content[:300], "...")

🟢 챗봇 답변:
빠르면 3학년부터 준비하는 경우도 있지만, 대부분 3학년 2학기부터 팀을 구한 후 4학년 때 프로젝트를 진행합니다. 논문의 경우도 대부분 4학년 때 준비합니다.

📚 유사 문서:

1. 유사도: 99.5571
- 96 -
 다. 인증 요건§선택 언어별 모든 레벨에서 최소 1과목(3학점) 이상 이수(영어는 1~4레벨)§총 5과목 수강(15학점), 수강과목 평점 3.5 이상 라. 시행시기§2020년 후기졸업자부터 인증서 발급 가능 마. 혜   택 §총장 명의 인증서 수여 바. 신청 및 수령 방법§매 학기말 홈페이지 공지사항 참고§참빛인재양성 통합관리 시스템 사이트 내 신청(성적증명서 첨부)§학위수여식(2월, 8월) 이후 인제니움대학 교학팀에서 수령 가능 ...

2. 유사도: 105.1390
정 되 었 습 니 다. 따 라 서  공 학 인 증 으 로  졸 업 하 는  학 생 들 은  필 수 로  이 수 하 여 야  합 니 다.2024학년도 입학자부터 적용구분영역교과목 명이수구분공학인증개설학기/학점1학년2학년3학년4학년1학기2학기1학기2학기1학기2학기1학기2학기교선(수학 및 기초과학 영역에서 12학점이수) ...

3. 유사도: 105.4390
- 49 -
(2016학번 입학자 적용) 졸업요건졸업이수학점 : 140학점(필수 포함)구분학과교양 전공졸업요건 학점최소 졸업요건 ‘공학필수’
전자정보공과대학
전자공학과
① 학번에 따른 「교양 교과목 이수체계」참고② 공학계열 단과대학 학과별 「기초교양 및 MSC(수학, 기초과학, 전산학)교과과정표」참고
전공 전필포함60학점(설계 12학점 포함)공학설계입문, 캡스톤설계전자통신공학과공학설계입문,,예비캡스톤설계, 캡스톤설계전자융합공학과공학설계입문, 캡스톤설계1전기공학과공학설계입문, 캡스톤설계전자재료공학과공학설계입문, 캡스톤설계1,캡스톤설계 ...


In [65]:
from langchain_community.vectorstores import Chroma

user_input = "졸업까지 최소 몇 학기 남았어?"

# Chroma DB 연결
db = Chroma(
    persist_directory="./chroma_db",
    collection_name="lecture_search",
    embedding_function=embeddings_model
)


response = answer_query(user_input, user_id="kim")
print("🟢 챗봇 답변:")
print(response)

similar_docs = db.similarity_search_with_score(user_input, k=3)

print("\n📚 유사 문서:")
for i, (doc, score) in enumerate(similar_docs, 1):
    print(f"\n{i}. 유사도: {score:.4f}")
    print(doc.page_content[:300], "...")

🟢 챗봇 답변:
이번 학기 포함 최소 4개의 학기가 남아있습니다.

📚 유사 문서:

1. 유사도: 89.2105
- 33 -
※ 지정과목은 반드시 졸업할 때까지 이수하여야 하며, 한 과목이라도 이수하지 않을 경우에는 졸업이 불가함. 라. 3학년 1학기 편입한 학생은 4학기 이상을 등록을 하여야 하고, 2학년 1학기에 편입한 학생은 6학기 이상을 등록하여야 함. 3학년 2학기 편입생은 편입학년의 1학기에 개설된 전공필수과목을 포함하여 모두 이수하여야 함       (예: 3학년 2학기 편입생 - 졸업 이전까지 3학년 1학기 전공필수과목을 필히 이수하여야 함).마. 편입생은 교양필수 이수를 면제하고, 교양학점 30학점을 이수하도록 함.6
다전공( ...

2. 유사도: 95.3136
수강신청 학점범위
가. 수강신청 학점 범위① 2016학번까지 : (최소)12학점 ~ (최대)21학점② 2017학번부터 : (최소)12학점 ~ (최대)19학점※ 졸업예정학기 (8학기, 건축학과 10학기)학생은 최소 수강신청 학점을 6학점으로 적용함.※ 8학기 (건축학과 10학기) 초과 수강신청자는 최저학점 (12학점) 미만으로 수강신청 가능함.※ 다전공, 재수강, 타 대학 학점교류 등의 학점은 최대 수강신청학점에 포함됨.※ 수강신청 학점이 미달(12학점 미만, 졸업예정학기 학생 6학점 미만)되지 않도록 유의할 것.③ 학점이월제도(20 ...

3. 유사도: 97.2750
- 28 -
바. 2016학년도 신입학자  1) 졸업이수학점(2017학년도 2학년 편입생, 2018학년도 3학년 편입생 포함)
   
단 과 대 학 / 학 과 교양전공(필수포함)졸업이수학점필수+균형기초전정공대, 공과대13~1624~3060140건축학과(5년제) 13~16-120170자연대수학과7~10-54140전자바이오물리학과,화학과, 생활체육학과13~16-60140정보콘텐츠학과(사이버정보보안학과)30-45120인문사회과학대학국어국문학과16~19-51140영어영문학과45140산업심리학과51140미디어영상학부45140동

In [66]:
from langchain_community.vectorstores import Chroma

user_input = "우리 학교에서 졸업 유예 신청이 가능해?"

# Chroma DB 연결
db = Chroma(
    persist_directory="./chroma_db",
    collection_name="lecture_search",
    embedding_function=embeddings_model
)


response = answer_query(user_input, user_id="kim")
print("🟢 챗봇 답변:")
print(response)

similar_docs = db.similarity_search_with_score(user_input, k=3)

print("\n📚 유사 문서:")
for i, (doc, score) in enumerate(similar_docs, 1):
    print(f"\n{i}. 유사도: {score:.4f}")
    print(doc.page_content[:300], "...")

🟢 챗봇 답변:
아니요. 우리 학교에서는 졸업 유예 신청이 불가능합니다.

📚 유사 문서:

1. 유사도: 83.2549
수 있도록 하는 제도2) 적용대상 : 2017학년도 신입생부터 / 시행시기 : 2017학년도 2학기부터3) 신청조건          가. 직전학기 평량평점이 3.5이상인 학생         나. 직전학기 평량평점이 산정되지 않는 인턴십, 교환학생 등을 이수한 학생은 해당사항 없음.         다. 해당학생은 수강신청 프로그램에서 추가 신청하면 됨.4) 2017학년도 이전 입학생은 아래의 학점이월제도를 유지함. ...

2. 유사도: 89.1385
후 신청서를 출력하여 7일 이내에 직접 교육지원팀을 방문해야 함3) 신청학점 및 학점인정 : 서울권역 e-러닝 신청학점을 포함하여 최대 6학점☞ 유의: ‘수강신청 학점범위’를 초과할 수 없으며, 졸업학점의 1/5선을 초과할 수 없음. 타학교 학점교류를 통하여 재수강이 불가함        가) 성적처리 : 해당교에서 취득한 성적을 백분율 점수 그대로 인정하되 등급은 본교 기준 성적등급으로 적용        나) 이수구분 인정          - 교류대학의 전공과목 : 일선(일반선택) /  교류대학의 교양과목 : 교선(교양선택)     ...

3. 유사도: 89.3383
기간 : 2월 25일(화) 10:00 ~ 15:00   ※ 소속학과 학생 수강신청 후 남은 여석에 한하여 신청 가능함.   ※ 본인 소속학과 전공 교과목 및 교양교과목도 여석이 있을 시 신청 가능.   ※ 2025학년도 1학기 2~4학년 재입학생, 편입학생(외국인포함) 신청 가능    라. 2월 26일(수) ~ 2월 28일(금) 10:00 ~ 15:00 : 한 과목도 신청하지 못한 재학생(편입학생 포함) 및 자과 수강신청 기간 이후 복학한 학생(기간 중 1일만 가능)  마. 수강신청변경 기간 : 3월 10일(월) ~ 3월 11일(화 ...


In [69]:
from langchain_community.vectorstores import Chroma

user_input = "조기 졸업이 가능한 상황이야?"

# Chroma DB 연결
db = Chroma(
    persist_directory="./chroma_db",
    collection_name="lecture_search",
    embedding_function=embeddings_model
)


response = answer_query(user_input, user_id="kim")
print("🟢 챗봇 답변:")
print(response)

similar_docs = db.similarity_search_with_score(user_input, k=3)

print("\n📚 유사 문서:")
for i, (doc, score) in enumerate(similar_docs, 1):
    print(f"\n{i}. 유사도: {score:.4f}")
    print(doc.page_content[:300], "...")

🟢 챗봇 답변:
조기 졸업 가능 조건은 6학기 말까지 114학점 이상 취득(계절수업 제외), 7개 학기 이상 이수자로 학칙시행세칙 제16조 평점평균의 산출에 따른 총 평점평균이 4.3 이상인 자(학적부성적 기준), 조기졸업 신청 당해 학기까지 취득한 전과목 성적이 B0 이상인 자(학적부성적 기준)입니다. 현재 C+ 이하의 성적이 존재하므로 재수강을 통해 성적을 개선하고 평점 평균을 4.3 이상으로 만든다면 조기 졸업이 가능합니다.

📚 유사 문서:

1. 유사도: 121.4522
- 8 -
※ 졸업예정학기(8학기, 건축학과 10학기) 학생이 수강포기를 하여도 총 신청학점이 6학점 이상이 될 경우에는 KLAS에서 직접 신청 가능함.4
수강신청 유의사항 및 기타 안내사항 ...

2. 유사도: 121.9826
- 34 -
공 기준 이상 취득한 후 중도 포기하면서 부전공을 신청할 경우에는 해당 전공을 부전공으로 인정함.2) 다전공을 신청한 학생이 졸업요건 및 졸업학점을 이수하였으나 다전공학점의 이수를 완료하지 못하여 졸업을 유보하고자 할 경우 졸업연기신청을 하여 다전공을 이수할 수 있음.   ※ 단, 마이크로전공은 제외.3) 다전공 신청 전에 해당 교과목을 이수하고 이후에 다전공을 신청하더라도 다전공 학점으로 인정됨.(졸업 사정시 반영)4) 2020학년도부터 공과대학 건축학과의 복수(부)전공 이수가 가능함.   (건축학과 → 타학과 / 타 ...

3. 유사도: 123.4763
또는 3.0/4.5이상(Lab인턴십     과정) 조건을 충족하지 못한 자  다. 대학원 입학 후 첫 학기는 휴학 및 자퇴를 신청할 수 없습니다.      (단, 군입대, 학업지속 불가의 질병 제외)  라. 학․석사 연계과정 선발자는 학부 졸업 시까지 군입대, 병가를 제외하고는 휴학을 신청할 수 없습니다.  마. 학부졸업요건을 충족한 자는 7학기말(공과대학 건축학과는 9학기말)에 학․석사 연계과정 학부 졸업신청서를 교무처 교육지원팀에, 대학원 입학지원서를 대학원 교학팀에 기한내에 제출하여야 

In [70]:
from langchain_community.vectorstores import Chroma

user_input = "지금 재수강가능한 과목들을 A0로 성적올리면 전체 학점이 어떻게 돼?"

# Chroma DB 연결
db = Chroma(
    persist_directory="./chroma_db",
    collection_name="lecture_search",
    embedding_function=embeddings_model
)


response = answer_query(user_input, user_id="kim")
print("🟢 챗봇 답변:")
print(response)

similar_docs = db.similarity_search_with_score(user_input, k=3)

print("\n📚 유사 문서:")
for i, (doc, score) in enumerate(similar_docs, 1):
    print(f"\n{i}. 유사도: {score:.4f}")
    print(doc.page_content[:300], "...")

🟢 챗봇 답변:
AI수학: A0 / 정보디자인프로그래밍: A0 → 예상 평점: 3.85

📚 유사 문서:

1. 유사도: 57.7915
재수강 제도 안내
가. 재수강이란 학점삭제 없이 기존에 이수한 동일교과목을 수강신청하여 이수할 수 있는 제도나. 재수강 조건 C+ 이하교과목 중 동일과목(으로 인정하는) 교과목을 2회에 한하여 재수강 가능다. 재수강 제한범위 재학 중 최대 8과목(단 F 또는 NP 학점은 과목 수에서 제외)라. 취득 성적 제한 재수강 후 취득성적 상한선은 A0 이하로 함마. 재수강한 과목의 표기 기 이수한 과목에 ‘R’(Retake)로 표기하고, 재수강한 과목에 성적 표기바. 취득학점 및 평점평균 산입방법 : ‘R’로 표기한 교과목(기 이수 과목)은 ...

2. 유사도: 79.4326
- 33 -
※ 지정과목은 반드시 졸업할 때까지 이수하여야 하며, 한 과목이라도 이수하지 않을 경우에는 졸업이 불가함. 라. 3학년 1학기 편입한 학생은 4학기 이상을 등록을 하여야 하고, 2학년 1학기에 편입한 학생은 6학기 이상을 등록하여야 함. 3학년 2학기 편입생은 편입학년의 1학기에 개설된 전공필수과목을 포함하여 모두 이수하여야 함       (예: 3학년 2학기 편입생 - 졸업 이전까지 3학년 1학기 전공필수과목을 필히 이수하여야 함).마. 편입생은 교양필수 이수를 면제하고, 교양학점 30학점을 이수하도록 함.6
다전공( ...

3. 유사도: 81.8579
수치해석, 이산수학, 통계학개론은 기초교양 → 전선(7000,I000,1000-)          그 외 교과목 기초교양 → 교선(0000-)      바. 다음의 경우는 수강신청이 되지 않도록 조치되어 있음(전산처리 불가)1) 신입생의 경우, 학번별로 지정한 본인 해당분반 과목이 아닌 타 분반 과목의 수강신청  2) 수업시간이 중복된 과목 / 제한인원이 초과된 과목 / 수강신청 학점범위를 초과 신청하는 경우 3) 기 이수한 동일한 과목을 수강신청 할 경우 C+이하 교과목은 재수강으로 처리되며, 

In [74]:
from langchain_community.vectorstores import Chroma

user_input = "지금 재수강가능한 과목은 뭐야?"

# Chroma DB 연결
db = Chroma(
    persist_directory="./chroma_db",
    collection_name="lecture_search",
    embedding_function=embeddings_model
)


response = answer_query(user_input, user_id="kim")
print("🟢 챗봇 답변:")
print(response)

similar_docs = db.similarity_search_with_score(user_input, k=3)

print("\n📚 유사 문서:")
for i, (doc, score) in enumerate(similar_docs, 1):
    print(f"\n{i}. 유사도: {score:.4f}")
    print(doc.page_content[:300], "...")

🟢 챗봇 답변:
지금 재수강 가능한 과목은 AI수학, 정보디자인프로그래밍입니다.

📚 유사 문서:

1. 유사도: 63.2386
재수강 제도 안내
가. 재수강이란 학점삭제 없이 기존에 이수한 동일교과목을 수강신청하여 이수할 수 있는 제도나. 재수강 조건 C+ 이하교과목 중 동일과목(으로 인정하는) 교과목을 2회에 한하여 재수강 가능다. 재수강 제한범위 재학 중 최대 8과목(단 F 또는 NP 학점은 과목 수에서 제외)라. 취득 성적 제한 재수강 후 취득성적 상한선은 A0 이하로 함마. 재수강한 과목의 표기 기 이수한 과목에 ‘R’(Retake)로 표기하고, 재수강한 과목에 성적 표기바. 취득학점 및 평점평균 산입방법 : ‘R’로 표기한 교과목(기 이수 과목)은 ...

2. 유사도: 81.9564
이를 재수강하는 학생은 ‘대학물리및실험2’ 재수강분반에서 이수* 2023학년도 2학기까지 MSC 내의 기초교양필수로 있었던 ‘확률및불규칙신호론’ 과목과 기초교양선택으로 있었던 ‘수치해석’, ‘이산수학’ 과목은 2024학년부터 전정대 공통으로 개설되는 전공 선택 과목으로 지정되었습니다. 2023학년도 2학기 이전까지 “확률 및 불규칙 신호론”과목 및 “수치해석”, “이산수학” 과목을 수강하신 학생들은 MSC 기초교양필수 및 기초교양선택으로 인정이 가능합니다.  2016학년도 이전 입학자 적용구분영역교과목 명이수구분개설학기/학점1학년2 ...

3. 유사도: 84.5869
- 72 -
는 재수강분반에서 이수 * 2022학년도1학기 ‘대학물리학1’ 기초과학 과목 신설에 따라, 2017학년도 입학자 적용 표에서 참고하여 이수함(실험학점 없음)* 2024학년도 1학기부터 ‘수치해석’ 및 ‘통계학개론’ 과목이 공과대 공통 전공선택으로 개설됨에 따라, 다음을 참고하여 이수함  - 2024학년도 1학기부터 개설되는 ‘수치해석’ 및 ‘통계학개론’은 전공선택으로 구분되며 수학, 기초과학, 전산학(MSC) 학점으로 인정  - 2023학년도 2학기까지 개설된 ‘수치해석’ 및 ‘통계학개론’은

In [75]:
from langchain_community.vectorstores import Chroma

user_input = "재수강 가능한 횟수가 어떻게 돼?"

# Chroma DB 연결
db = Chroma(
    persist_directory="./chroma_db",
    collection_name="lecture_search",
    embedding_function=embeddings_model
)


response = answer_query(user_input, user_id="kim")
print("🟢 챗봇 답변:")
print(response)

similar_docs = db.similarity_search_with_score(user_input, k=3)

print("\n📚 유사 문서:")
for i, (doc, score) in enumerate(similar_docs, 1):
    print(f"\n{i}. 유사도: {score:.4f}")
    print(doc.page_content[:300], "...")

🟢 챗봇 답변:
김브티님은 총 8번까지 재수강이 가능하며, 현재까지 1번 재수강하셨습니다. 따라서 7번 더 재수강할 수 있습니다.

📚 유사 문서:

1. 유사도: 68.4382
재수강 제도 안내
가. 재수강이란 학점삭제 없이 기존에 이수한 동일교과목을 수강신청하여 이수할 수 있는 제도나. 재수강 조건 C+ 이하교과목 중 동일과목(으로 인정하는) 교과목을 2회에 한하여 재수강 가능다. 재수강 제한범위 재학 중 최대 8과목(단 F 또는 NP 학점은 과목 수에서 제외)라. 취득 성적 제한 재수강 후 취득성적 상한선은 A0 이하로 함마. 재수강한 과목의 표기 기 이수한 과목에 ‘R’(Retake)로 표기하고, 재수강한 과목에 성적 표기바. 취득학점 및 평점평균 산입방법 : ‘R’로 표기한 교과목(기 이수 과목)은 ...

2. 유사도: 98.5147
수치해석, 이산수학, 통계학개론은 기초교양 → 전선(7000,I000,1000-)          그 외 교과목 기초교양 → 교선(0000-)      바. 다음의 경우는 수강신청이 되지 않도록 조치되어 있음(전산처리 불가)1) 신입생의 경우, 학번별로 지정한 본인 해당분반 과목이 아닌 타 분반 과목의 수강신청  2) 수업시간이 중복된 과목 / 제한인원이 초과된 과목 / 수강신청 학점범위를 초과 신청하는 경우 3) 기 이수한 동일한 과목을 수강신청 할 경우 C+이하 교과목은 재수강으로 처리되며, B0까지의 교과목은 수강신청 되지 않 ...

3. 유사도: 104.3101
기간 : 2월 25일(화) 10:00 ~ 15:00   ※ 소속학과 학생 수강신청 후 남은 여석에 한하여 신청 가능함.   ※ 본인 소속학과 전공 교과목 및 교양교과목도 여석이 있을 시 신청 가능.   ※ 2025학년도 1학기 2~4학년 재입학생, 편입학생(외국인포함) 신청 가능    라. 2월 26일(수) ~ 2월 28일(금) 10:00 ~ 15:00 : 한 과목도 신청하지 못한 재학생(편입학생 포함) 및 자과 수강신청 기간 이후 복학한 학생(기간 중

In [76]:
from langchain_community.vectorstores import Chroma

user_input = "같은 과목을 재수강할 때 교수님이 바뀌거나 과목이 사라지면 어떻게 해야 해?"

# Chroma DB 연결
db = Chroma(
    persist_directory="./chroma_db",
    collection_name="lecture_search",
    embedding_function=embeddings_model
)


response = answer_query(user_input, user_id="kim")
print("🟢 챗봇 답변:")
print(response)

similar_docs = db.similarity_search_with_score(user_input, k=3)

print("\n📚 유사 문서:")
for i, (doc, score) in enumerate(similar_docs, 1):
    print(f"\n{i}. 유사도: {score:.4f}")
    print(doc.page_content[:300], "...")

🟢 챗봇 답변:
학정번호 기준으로 동일한 학정번호가 있는 강의를 수강하시면 됩니다. 다만, 동일한 학정번호의 강의가 3년 이상 미개설 할 경우, C이하의 과목은 학점삭제 가능합니다. 자세한 내용은 학사에 문의하세요.

📚 유사 문서:

1. 유사도: 72.3925
재수강 제도 안내
가. 재수강이란 학점삭제 없이 기존에 이수한 동일교과목을 수강신청하여 이수할 수 있는 제도나. 재수강 조건 C+ 이하교과목 중 동일과목(으로 인정하는) 교과목을 2회에 한하여 재수강 가능다. 재수강 제한범위 재학 중 최대 8과목(단 F 또는 NP 학점은 과목 수에서 제외)라. 취득 성적 제한 재수강 후 취득성적 상한선은 A0 이하로 함마. 재수강한 과목의 표기 기 이수한 과목에 ‘R’(Retake)로 표기하고, 재수강한 과목에 성적 표기바. 취득학점 및 평점평균 산입방법 : ‘R’로 표기한 교과목(기 이수 과목)은 ...

2. 유사도: 78.7471
- 70 -
  - 2017학년도까지 ‘컴퓨터그래픽’을 미이수한 경우, 2018학년도부터 필수교양(교필)으로 개설되는 ‘인공지능과컴퓨팅사고(구.컴퓨팅사고)’를 대체하여 이수  - 2016학년도까지 ‘컴퓨터활용’ 교과목을 이수하였으나, 이를 재수강하는 학생은 2020학년도까지 개설되는 재수강분반에서 이수* 2024학년도 1학기부터 ‘수치해석’ 및 ‘통계학개론’ 과목이 공과대 공통 전공선택으로 개설됨에 따라 다음을 참고하여 이수함  - 2024학년도 1학기부터 개설되는 ‘수치해석’ 및 ‘통계학개론’은 전공선택으로 구분되며 수학, 기초과 ...

3. 유사도: 78.8133
수치해석, 이산수학, 통계학개론은 기초교양 → 전선(7000,I000,1000-)          그 외 교과목 기초교양 → 교선(0000-)      바. 다음의 경우는 수강신청이 되지 않도록 조치되어 있음(전산처리 불가)1) 신입생의 경우, 학번별로 지정한 본인 해당분반 과목이 아닌 타 분반 과목의 수강신청  2) 수업시간이 중복된 과목 / 제한인원이 초과된 과목 /

In [77]:
from langchain_community.vectorstores import Chroma

user_input = "재수강하면 성적표에 재수강 여부가 표시되나?"

# Chroma DB 연결
db = Chroma(
    persist_directory="./chroma_db",
    collection_name="lecture_search",
    embedding_function=embeddings_model
)


response = answer_query(user_input, user_id="kim")
print("🟢 챗봇 답변:")
print(response)

similar_docs = db.similarity_search_with_score(user_input, k=3)

print("\n📚 유사 문서:")
for i, (doc, score) in enumerate(similar_docs, 1):
    print(f"\n{i}. 유사도: {score:.4f}")
    print(doc.page_content[:300], "...")

🟢 챗봇 답변:
재수강한 과목은 성적표에 ‘R’(Retake)로 표기되며, 재수강한 과목의 성적만 성적 및 학점에 반영됩니다.

📚 유사 문서:

1. 유사도: 80.4364
재수강 제도 안내
가. 재수강이란 학점삭제 없이 기존에 이수한 동일교과목을 수강신청하여 이수할 수 있는 제도나. 재수강 조건 C+ 이하교과목 중 동일과목(으로 인정하는) 교과목을 2회에 한하여 재수강 가능다. 재수강 제한범위 재학 중 최대 8과목(단 F 또는 NP 학점은 과목 수에서 제외)라. 취득 성적 제한 재수강 후 취득성적 상한선은 A0 이하로 함마. 재수강한 과목의 표기 기 이수한 과목에 ‘R’(Retake)로 표기하고, 재수강한 과목에 성적 표기바. 취득학점 및 평점평균 산입방법 : ‘R’로 표기한 교과목(기 이수 과목)은 ...

2. 유사도: 103.8518
- 17 -
사. ‘과목 선택’에 조회된 과목을 노란색 ‘수강신청’ 버튼을 클릭한 후 재수강 과목 알림 팝업창에 ‘확인’을 클릭합니다. 
아. 수강신청 완료시 ‘종료’ 클릭 후 본인의 수강내역 및 시간표 정보를 KLAS에서 반드시 확인바랍니다. ...

3. 유사도: 110.4400
편입생이 이수해야 할 교과목 및 학점
가. 편입생은 수강신청을 하기 전에 반드시 소속 학과(부)장과 면담하여 전적교에서 취득한 학점을 인정받기 위한 절차를 거쳐 인정받은 학점(교양․ 전공․ 일선)과 향후 반드시 이수해야 할 교과목을 확인하고 수강 신청하여야 함.   ※ 해당 학과에서 학점 심의를 통해 학점을 인정하며, 인정된 학점 및 이수요망 교과목에 대하여는 추후 공지를 통하여 배부되며, 학교홈페이지 KLAS>수강성적조회 메뉴에서도 확인 가능. 나. 전적교에서 이수한 교과목과 동일한 과목을 본교에서 다시 이수하여 학점을 취득할 수 ...


In [78]:
from langchain_community.vectorstores import Chroma

user_input = "F받고 재수강하면 재수강 이력이 남아?"

# Chroma DB 연결
db = Chroma(
    persist_directory="./chroma_db",
    collection_name="lecture_search",
    embedding_function=embeddings_model
)


response = answer_query(user_input, user_id="kim")
print("🟢 챗봇 답변:")
print(response)

similar_docs = db.similarity_search_with_score(user_input, k=3)

print("\n📚 유사 문서:")
for i, (doc, score) in enumerate(similar_docs, 1):
    print(f"\n{i}. 유사도: {score:.4f}")
    print(doc.page_content[:300], "...")

🟢 챗봇 답변:
F 받은 과목은 재수강 시 '재수강' 표시가 뜨지 않지만, 재수강 이력은 남고 성적표에는 'R'로 표기됩니다.

📚 유사 문서:

1. 유사도: 88.3057
재수강 제도 안내
가. 재수강이란 학점삭제 없이 기존에 이수한 동일교과목을 수강신청하여 이수할 수 있는 제도나. 재수강 조건 C+ 이하교과목 중 동일과목(으로 인정하는) 교과목을 2회에 한하여 재수강 가능다. 재수강 제한범위 재학 중 최대 8과목(단 F 또는 NP 학점은 과목 수에서 제외)라. 취득 성적 제한 재수강 후 취득성적 상한선은 A0 이하로 함마. 재수강한 과목의 표기 기 이수한 과목에 ‘R’(Retake)로 표기하고, 재수강한 과목에 성적 표기바. 취득학점 및 평점평균 산입방법 : ‘R’로 표기한 교과목(기 이수 과목)은 ...

2. 유사도: 117.0266
- 64 -
  - 2016학년도까지 ‘C프로그래밍’을 미이수한 경우, 2017학년도부터 필수교양(교필)으로 개설되는 ‘C프로그래밍’으로 대체하여 이수  - 2016학년도까지 ‘컴퓨터개론’을 미이수한 경우, 2017학년도부터 필수교양(교필)으로 개설되는 ‘인공지능과컴퓨팅사고(구.컴퓨팅사고)’로        대체하여 이수  - 2016학년도까지 ‘컴퓨터개론’ 교과목을 이수하였으나, 이를 재수강하는 학생은 2020학년도까지 개설되는 재수강분반에서 이수  - 2016학년도까지 ‘고급C프로그래밍및설계’를 미이수한 경우, 2017학년도부터  ...

3. 유사도: 120.8467
- 75 -
마. 선ž후수 교과목 이수체계 준수 제도       1) 선수 지정 교과목을 미이수 한 학생은 담당과목교수의 상담(Oral Test, Quiz등)을 통해 「선수과목 미 이수 인정(승인)심사」를 받을 수 있으며, 기한 내에 ‘승인’ 받은 학생은 수강가능함. 단, 승인을 받지 못한 학생은 해당 수강신청 과목이 ‘수강삭제조치(일괄)’되어 해당 학기에 수강할 수 없음.   2) 학과 공학프로그램별 교과목 이수체계도 : 학과 홈페이지 안내 참조   3

In [79]:
from langchain_community.vectorstores import Chroma

user_input = "성적이 낮은 전공 필수 과목을 재수강하는 게 나을까, 다른 전공 과목을 듣는 게 나을까?"

# Chroma DB 연결
db = Chroma(
    persist_directory="./chroma_db",
    collection_name="lecture_search",
    embedding_function=embeddings_model
)


response = answer_query(user_input, user_id="kim")
print("🟢 챗봇 답변:")
print(response)

similar_docs = db.similarity_search_with_score(user_input, k=3)

print("\n📚 유사 문서:")
for i, (doc, score) in enumerate(similar_docs, 1):
    print(f"\n{i}. 유사도: {score:.4f}")
    print(doc.page_content[:300], "...")

🟢 챗봇 답변:
재수강 여부는 개인의 학업 목표와 상황에 따라 다르지만, 성적이 낮은 전공 과목을 재수강하는 것은 전공에 대한 이해도를 높이고 평점 평균을 개선하는 데 도움이 될 수 있습니다. 반면, 다른 전공 과목을 듣는다면 다양한 지식을 쌓을 수 있는 기회가 될 수 있습니다. 현재 성적을 고려할 때, 재수강이 가능한 전공 과목으로는 C+를 받은 AI수학 과목 / C+를 받은 정보디자인프로그래밍 과목이 있습니다.

📚 유사 문서:

1. 유사도: 67.2534
재수강 제도 안내
가. 재수강이란 학점삭제 없이 기존에 이수한 동일교과목을 수강신청하여 이수할 수 있는 제도나. 재수강 조건 C+ 이하교과목 중 동일과목(으로 인정하는) 교과목을 2회에 한하여 재수강 가능다. 재수강 제한범위 재학 중 최대 8과목(단 F 또는 NP 학점은 과목 수에서 제외)라. 취득 성적 제한 재수강 후 취득성적 상한선은 A0 이하로 함마. 재수강한 과목의 표기 기 이수한 과목에 ‘R’(Retake)로 표기하고, 재수강한 과목에 성적 표기바. 취득학점 및 평점평균 산입방법 : ‘R’로 표기한 교과목(기 이수 과목)은 ...

2. 유사도: 71.2048
- 70 -
  - 2017학년도까지 ‘컴퓨터그래픽’을 미이수한 경우, 2018학년도부터 필수교양(교필)으로 개설되는 ‘인공지능과컴퓨팅사고(구.컴퓨팅사고)’를 대체하여 이수  - 2016학년도까지 ‘컴퓨터활용’ 교과목을 이수하였으나, 이를 재수강하는 학생은 2020학년도까지 개설되는 재수강분반에서 이수* 2024학년도 1학기부터 ‘수치해석’ 및 ‘통계학개론’ 과목이 공과대 공통 전공선택으로 개설됨에 따라 다음을 참고하여 이수함  - 2024학년도 1학기부터 개설되는 ‘수치해석’ 및 ‘통계학개론’은 전공선택으로 구분되며 수학, 기초과 ...

3. 유사도: 71.4880
- 72 -
는 재수강분반에서 이수 * 2022학년도1학기 ‘대학물리학1’ 기초과학 과목 신설에 따라, 2017학년도 입학자 적용 표에서 참고하여 이수함(실험학점 없음)*

###### 강의탐색

In [71]:
from langchain_community.vectorstores import Chroma

user_input = "컴퓨터그래픽스 강의는 시험이 오픈북이야?"

# Chroma DB 연결
db = Chroma(
    persist_directory="./chroma_db",
    collection_name="lecture_search",
    embedding_function=embeddings_model
)


response = answer_query(user_input, user_id="kim")
print("🟢 챗봇 답변:")
print(response)

similar_docs = db.similarity_search_with_score(user_input, k=3)

print("\n📚 유사 문서:")
for i, (doc, score) in enumerate(similar_docs, 1):
    print(f"\n{i}. 유사도: {score:.4f}")
    print(doc.page_content[:300], "...")

🟢 챗봇 답변:
컴퓨터그래픽스의 시험은 강의 후기 결과 "Closed book"로 확인 됩니다. 

📚 유사 문서:

1. 유사도: 31.2182
질문: 컴퓨터그래픽스 강의는 시험이 오픈북이야?
답변: 컴퓨터그래픽스의 시험은 강의 후기 결과 "Closed book"로 확인 됩니다. ...

2. 유사도: 61.3945
질문: 딥러닝프로그래밍 강의는 시험이 오픈북이야?
답변: 딥러닝프로그래밍의 시험은 강의 후기 결과 "Closed book"로 확인 됩니다. ...

3. 유사도: 82.2889
학정번호: I040-3-5474-02
강의명: 컴퓨터비전
강의평점: 
과제: 보통
팀플: 없음
성적평가정도: 보통
출결 방식: 전자출결
시험 횟수: 2
시험 방식: Open book
전공 학점: 없음
교양 학점: 없음
총 학점: 없음
교수명: 
수업 시간: 
강의 유형: 
학점: 시간
학기: 학기
강의 설명: 영역: ...


In [72]:
from langchain_community.vectorstores import Chroma

user_input = "UX/UI디자인 강의는 성적 평가 기준이 어떻게 돼? 시험, 과제, 출석 비율 알려줘."

# Chroma DB 연결
db = Chroma(
    persist_directory="./chroma_db",
    collection_name="lecture_search",
    embedding_function=embeddings_model
)


response = answer_query(user_input, user_id="kim")
print("🟢 챗봇 답변:")
print(response)

similar_docs = db.similarity_search_with_score(user_input, k=3)

print("\n📚 유사 문서:")
for i, (doc, score) in enumerate(similar_docs, 1):
    print(f"\n{i}. 유사도: {score:.4f}")
    print(doc.page_content[:300], "...")

🟢 챗봇 답변:
UX/UI디자인 강의는 출석 10, 중간고사 40, 기말고사 30, 과제보고서 10, 수업태도 10으로 구성되어 있습니다.

📚 유사 문서:

1. 유사도: 16.1193
질문: UX/UI디자인 강의는 성적 평가 기준이 어떻게 돼? 시험, 과제, 출석 비율 알려줘.
답변: UX/UI디자인 강의는 출석 10, 중간고사 40, 기말고사 30, 과제보고서 10, 수업태도 10으로 구성되어 있습니다. ...

2. 유사도: 40.2832
질문: UX/UI디자인 강의 평가에서 성적평가가 엄격하다는 의견이 많아?
답변: UX/UI디자인 강의 성적평가 기준은 '보통'입니다. ...

3. 유사도: 53.0354
질문: UXUI 디자인 강의 수업 방식은 어떤 식으로 진행돼? 발표가 많은 편이야?
답변: 강의 후기에서는 주제발표와 최종 프로젝트 발표 두번 이뤄집니다. 자세한 내용은 교수님께 문의 하시길 바랍니다. ...


###### 진로상담

In [73]:
from langchain_community.vectorstores import Chroma

user_input = "학교 수업들 중에 협업 경험을 쌓을 수 있게 팀프로젝트를 진행하는 수업 추천해줘"

# Chroma DB 연결
db = Chroma(
    persist_directory="./chroma_db",
    collection_name="lecture_search",
    embedding_function=embeddings_model
)


response = answer_query(user_input, user_id="kim")
print("🟢 챗봇 답변:")
print(response)

similar_docs = db.similarity_search_with_score(user_input, k=3)

print("\n📚 유사 문서:")
for i, (doc, score) in enumerate(similar_docs, 1):
    print(f"\n{i}. 유사도: {score:.4f}")
    print(doc.page_content[:300], "...")

🟢 챗봇 답변:
기계학습, 데이터마이닝, 웹서비스설계및실습, 정보디자인프로그래밍, 텍스트마이닝 등 수업에서 팀프로젝트를 경험해 볼 수 있습니다.

📚 유사 문서:

1. 유사도: 64.5527
- 실습은 Python 언어를 기반으로 오픈소스 라이브러리를 활용하여 데이터를 다루고 알고리즘을 구현하며 활용능력을 배양합니다.
- 팀기반 학습(Team-Based Learning)으로 운영하기에, 1)온라인 강의 선수강, 2) 대면강의 시 퀴즈와 과제수행, 그리고 사례연구를 통해 소규모 팀참여로 참여하게 됩니다. ...

2. 유사도: 67.8428
질문: UX/UI디자인 강의 팀프로젝트가 있어?
답변: UX/UI디자인 수업은 강의 후기 결과 "조별과제 많음"으로 확인 됩니다. ...

3. 유사도: 71.0412
- 82 -
11
다학년 다학기 프로젝트(KW-Vertically Integrated Project) 교과목 안내
 가. 다학년 다학기 프로젝트(KW-VIP) 교과목이란?    ▪ 교수가 제시한 특정 연구주제에 관한 장기 프로젝트 중심 수업에 참여하여       다학기 동안  (※ 2학년 1학기부터 4학년 2학기까지 총 6회까지 수강이 가능합니다)       다학년, 다전공 학생들이 팀을 이루어 연구활동을 수행하는 정규 수업입니다.    ▪ 다학기 동안 동일 주제의 연구 프로젝트 수업에 참여하는 것을 권장하고 있으나,         ...
